In [1]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List, Optional
from pathlib import Path

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
from IPython.display import display, HTML

conf_path = str(Path("./") / settings.CONF_SOURCE)
conf_loader = OmegaConfigLoader(conf_source=conf_path)
credentials = conf_loader["credentials"]

## Define Knowledge Graph Schema

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional


class Concept(BaseModel):
    """
    A theoretical idea or opinion that composes the overall author's thesis.
    There are three types of concepts:
    - thesis: a high-level concept that synthesises the author 
    """

    id: int = Field(description="Unique identifier for the concept.")
    summary: str = Field(description="One sentence length, self-contained, summary of the concept.")
    title: str = Field(description="Title of the concept.")


class Reference(BaseModel):
    """
    An external source that supports the author's arguments.
    """

    id: int = Field(description="Unique identifier for the evidence.")
    summary: str = Field(
        description="One-sentence summary of the factual information."
    )
    title: str = Field(description="Title of the evidence.")


class ConceptLink(BaseModel):
    """
    Represents a directed link connecting a parent concept to a child concept.
    """

    parent_id: int = Field(description="Unique identifier of the parent concept.")
    child_id: int = Field(description="Unique identifier of the child concept.")


class EvidenceLink(BaseModel):
    """
    Represents a directed link connecting a leaf concept to a piece of evidence.
    """

    concept_id: int = Field(
        description="Unique identifier of the concept supported by the evidence."
    )
    evidence_id: int = Field(
        description="Unique identifier of the evidence supporting the concept."
    )


class EssayGraph(BaseModel):
    """
    Knowledge graph representing the content of an essay as a hierarchical tree connecting concepts and evidence.
    From the top level concepts that synthesise the author's thesis to the lowest level concepts that
    are the building blocks of the argumentation. Intermediate concepts are formed by splitting high level concepts into
    several sub concepts, allowing a granular representation of the author's argumentation.
    Lowest level concepts are connected to the evidence supporting them.
    """

    concepts: List[Concept] = Field(description="List of concepts in the essay.")
    evidence: List[Evidence] = Field(
        description="List of pieces of evidence in the essay."
    )
    concept_links: List[ConceptLink] = Field(
        description="List of links connecting low level concepts to their parent concepts."
    )
    evidence_links: List[Optional[EvidenceLink]] = Field(
        description="List of links connecting concepts to their supporting evidence in the essay."
    )

In [134]:
example_essay = """
Renewable energy is essential for sustainable development and environmental protection.As the world faces the consequences of climate change, the shift from fossil
fuels to renewable energy sources like wind, solar, and hydroelectric power is critical.
Renewable energy significantly reduces greenhouse gas emissions. Fossil fuels, when burned, release a large amount of carbon dioxide, a major contributor to global warming.
In contrast, wind and solar power generate electricity without emitting greenhouse gases. According to a 2020 study by the International Renewable Energy Agency (IRENA),
countries that invest in renewable energy can reduce their carbon footprint by up to 70%. Renewable energy decreases dependency on finite resources.
Fossil fuels are limited and will eventually run out, leading to energy crises and economic instability. Renewable energy sources, on the other hand, are abundant and can be replenished naturally. The sun provides more energy in an hour than the world uses in a year, making solar power a highly sustainable option.
Also energy promotes economic growth. The renewable energy sector creates jobs and stimulates technological innovation. For instance, the U.S. 
Bureau of Labor Statistics reports that the solar industry employed more than 240,000 people in 2020, with job growth expected to continue.
Furthermore, investments in renewable energy infrastructure can drive economic development, particularly in rural areas where many renewable energy projects are located.
In conclusion, the transition to renewable energy is crucial for mitigating climate change, ensuring energy security, and fostering economic prosperity. Governments, businesses, and individuals must collaborate to accelerate the adoption of renewable energy technologies and build a sustainable future.
"""

In [135]:
example_tree = """
{
  "concepts": [
    {
      "id": 1,
      "title": "The rise of renewable energy",
      "description": "Renewable energy is essential for sustainable development and environmental protection."
    },
    {
      "id": 2,
      "title": "Reduces Greenhouse Gas Emissions",
      "description": "Renewable energy significantly reduces greenhouse gas emissions."
    },
    {
      "id": 3,
      "title": "Decreases Dependency on Finite Resources",
      "description": "Renewable energy decreases dependency on finite resources."
    },
    {
      "id": 4,
      "title": "Promotes Economic Growth",
      "description": "Renewable energy promotes economic growth."
    }
  ],
  "evidence": [
    {
      "id": 1,
      "title": "IRENA Study",
      "description": "A 2020 study by the International Renewable Energy Agency (IRENA) shows that countries investing in renewable energy can reduce their carbon footprint by up to 70%."
    },
    {
      "id": 2,
      "title": "Sun Energy",
      "description": "The sun provides more energy in an hour than the world uses in a year, making solar power a highly sustainable option."
    },
    {
      "id": 3,
      "title": "Solar Industry Employment",
      "description": "The U.S. Bureau of Labor Statistics reports that the solar industry employed more than 240,000 people in 2020, with job growth expected to continue."
    }
  ],
  "concept_links": [
    {
      "parent_id": 1,
      "child_id": 2
    },
    {
      "parent_id": 1,
      "child_id": 3
    },
    {
      "parent_id": 1,
      "child_id": 4
    }
  ],
  "supported_by": [
    {
      "concept_id": 2,
      "evidence_id": 1
    },
    {
      "concept_id": 3,
      "evidence_id": 2
    },
    {
      "concept_id": 4,
      "evidence_id": 3
    }
  ]
}

"""

## Build generation chain

In [182]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.output_parsers import OutputFixingParser

parser = PydanticOutputParser(pydantic_object=EssayGraph)

chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=credentials["google_api_key"],
    generation_config={"response_mime_type": "application/json"},
    temperature=0.2,
)

fix_parser = OutputFixingParser.from_llm(parser=parser, llm=chat)

prompt = PromptTemplate(
    template="""

    <TASK DESCRIPTION>
    You are an highly skilled information extraction specialist, with high abstraction capabilities.
    Your task is to read carefully the given essay, analyse its content and synthetise it into an atomistic knowledge graph.
    Follow the following format instructions.
     {format_instructions}
    <TASK DESCRIPTION>

    <Essay>
    {text}
    </Essay>

    MANDATORY INSTRUCTION:
    The overall tree structure must present a coherent and logical informational flow from evidence to core ideas.
    Concept summaries must be self-contained, allowing understanding without surrounding context.
    Use the same writing perspective as the author. Do not add any personal opinion or interpretation.
    Do not use formulation such as "The author argues that", "The author claims that", "The essay states that", etc.
    """,
    input_variables=["text"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)


def invoke(text: str):
    chain = prompt | chat
    output = chain.invoke(dict(text=text))
    try:
        return parser.parse(output.content)
    except Exception as e:
        print(f"Parsing error: {e}, falling back to the fix_parser")
        return fix_parser.parse(output.content)

## Load sample input

In [186]:
chunks_df = catalog.load("chunks")
chunks = chunks_df.iloc[0:170]["text"].to_list()
# Concat chunks with next line separator
text = "\n".join(chunks)

# Save the text to a file
text_path = Path("data/01_raw/text.txt")
text_path.write_text(text)

[07/01/24 11:49:14] INFO     Loading data from chunks (ParquetDataset)...                       ]8;id=781867;file:///home/blollivier/.pyenv/versions/3.10.10/envs/book-summarizer/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=871161;file:///home/blollivier/.pyenv/versions/3.10.10/envs/book-summarizer/lib/python3.10/site-packages/kedro/io/data_catalog.py#508\508]8;;\

185333

### Invoke the model

In [187]:
kg = invoke({"text": text})

### Plot the resulting graph

In [11]:
from graphviz import Digraph, Graph


def create_knowledge_graph(essay_graph: dict):
    # Load the knowledge graph from the JSON file

    # Create a new graphviz Digraph object
    dot = Digraph("Essay Knowledge Graph", format="png")

    # Configure graph attributes
    configure_graph(dot)

    # Add nodes and edges to the graph
    add_nodes(dot, essay_graph)
    add_edges(dot, essay_graph)

    # Render and save the graph
    dot.render("essay_knowledge_graph", view=True)

def configure_graph(dot):
    # Set graph attributes for horizontal layout
    dot.attr(rankdir="LR", ranksep="1.5", nodesep="0.5")

    # Set default node and edge attributes
    dot.attr("node", fontname="Arial", shape="box", style="rounded,filled", margin="0.2")
    dot.attr("edge", fontname="Arial", fontsize="10")

def add_nodes(dot, essay_graph):
    node_types = {
        "concepts": ("#FFCC99", "C"),  # Light orange
        "evidence": ("#CCFFCC", "E")   # Light green
    }

    for node_type, (color, prefix) in node_types.items():
        with dot.subgraph(name=f"cluster_{node_type}") as c:
            c.attr(rankdir="TB")
            c.attr(label=node_type.capitalize(), style="rounded", color=color)
            for item in essay_graph[node_type]:
                c.node(
                    f"{prefix}{item['id']}",
                    f"<<b>{item['title']}</b><br/>{item['summary']}>",
                    fillcolor=color
                )

def add_edges(dot, essay_graph):
    # Add concept links
    for link in essay_graph["concept_links"]:
        dot.edge(
            f"C{link['parent_id']}",
            f"C{link['child_id']}",
            color="#FF9933"  # Orange
        )

    # Add supported_by links
    for link in essay_graph["evidence_links"]:
        dot.edge(
            f"C{link['concept_id']}",  # Note: 'argument_id' now refers to a concept
            f"E{link['evidence_id']}",
            color="#339933"  # Green
        )



In [ ]:
# Call the function to create the knowledge graph
essay_graph = catalog.load("knowledge_graph")
create_knowledge_graph(kg.dict())

## Use cluster information to generate the graph

In [7]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.output_parsers import OutputFixingParser

parser = PydanticOutputParser(pydantic_object=EssayGraph)

chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=credentials["google_api_key"],
    generation_config={"response_mime_type": "application/json"},
    temperature=0.
)

# chat = ChatOpenAI(
#     model="gpt-4o",
#     api_key=credentials["openai_api_key"],
#     response_format={"type": "json_object" },
#     temperature=0.2,
# )

fix_parser = OutputFixingParser.from_llm(parser=parser, llm=chat)

prompt = PromptTemplate(
    template="""

    <TASK DESCRIPTION>
    You are an highly skilled information extraction specialist, with high abstraction capabilities.
    You are given an essay, split into paragraphs. Each paragraph is already assigned to a cluster.

    Your task is: given the paragraphs text and the assigned cluster, build a knowledge graph.
    Follow the following format instructions.
     {format_instructions}
    <TASK DESCRIPTION>

    <PARAGRAPHS>
    {chunks}
    </PARAGRAPHS>

    MANDATORY INSTRUCTION:
    The clusters are the results of a previous analysis. They show which paragraphs are conceptually related.
    Use this information to extract the concepts. If the cluster is too small, consider merging it with another cluster.
    Cluster -1 indicates that the paragraph has not been assigned to a cluster, merge it with the closest cluster.
    
    The overall tree structure must present a coherent and logical informational flow from evidence to core ideas.
    Concept summaries must be self-contained, allowing understanding without surrounding context.
    Use the same writing perspective as the author. Do not add any personal opinion or interpretation.
    Do not use formulation such as "The author argues that", "The author claims that", "The essay states that", etc.
    """,
    input_variables=["chunks"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)


def invoke(chunks: str):
    chain = prompt | chat
    output = chain.invoke(dict(chunks=chunks))
    try:
        return parser.parse(output.content)
    except Exception as e:
        print(f"Parsing error: {e}, falling back to the fix_parser")
        return fix_parser.parse(output.content)

In [8]:
chunks = '[{"text": "La r\\u00e9ussite nourrit l\\u2019ambition, et nos r\\u00e9alisations r\\u00e9centes poussent\\n                            l\\u2019humanit\\u00e9 \\u00e0 se donner des objectifs encore plus audacieux.", "cluster": -1}, {"text": "Avec les niveaux de prosp\\u00e9rit\\u00e9, de sant\\u00e9 et d\\u2019harmonie sans pr\\u00e9c\\u00e9dent que\\n                            nous avons atteints, et compte tenu de notre bilan et de nos valeurs\\n                            actuelles, l\\u2019humanit\\u00e9 se fixera probablement comme prochainsobjectifs l\\u2019immortalit\\u00e9, le bonheur et la divinit\\u00e9.\\n                            Apr\\u00e8s avoir r\\u00e9duit la mortalit\\u00e9 li\\u00e9e \\u00e0 la faim, \\u00e0 la maladie et \\u00e0 la\\n                            violence, nous allons maintenant chercher \\u00e0 triompher de la vieillesse\\n                            et de la mort elle-m\\u00eame. Apr\\u00e8s avoir sauv\\u00e9 les gens de la mis\\u00e8re\\n                            profonde, nous allons chercher \\u00e0 les rendre vraiment heureux. Et ayant\\n                            sorti l\\u2019humanit\\u00e9 de la brutalit\\u00e9 des luttes pour la survie, nous allons\\n                            chercher \\u00e0 hisser les hommes au rang de dieux, \\u00e0 transformerHomo\\n                                sapiensenHomo deus.", "cluster": 258}, {"text": "AuXXIesi\\u00e8cle, les hommes risquent fort de viser\\n                            l\\u2019immortalit\\u00e9. La bataille contre la vieillesse et la mort ne fera que\\n                            prolonger le v\\u00e9n\\u00e9rable combat contre la famine et la maladie, en\\n                            illustrant la valeur supr\\u00eame de la culture contemporaine\\u00a0: la vie\\n                            humaine. On ne cesse de nous rappeler que la vie humaine est ce qu\\u2019il y\\n                            a de plus sacr\\u00e9 dans l\\u2019univers. Tout le monde le dit\\u00a0: les\\n                            enseignants dans les \\u00e9coles, les hommes politiques au parlement, les\\n                            avocats \\u00e0 la cour et les com\\u00e9diens sur sc\\u00e8ne. La D\\u00e9claration universelle\\n                            des droits de l\\u2019homme adopt\\u00e9e par les Nations unies au lendemain de la\\n                            derni\\u00e8re guerre \\u2013\\u00a0qui est ce qui ressemble sans doute le plus \\u00e0 une\\n                            constitution mondiale\\u00a0\\u2013 d\\u00e9clare cat\\u00e9goriquement que le \\u00ab\\u00a0droit \\u00e0\\n                            la vie\\u00a0\\u00bb est la valeur la plus fondamentale de l\\u2019humanit\\u00e9. Puisque\\n                            la mort viole clairement ce droit, la mort est un crime contre\\n                            l\\u2019humanit\\u00e9. Nous devons mener contre elle une guerre totale.", "cluster": 247}, {"text": "Tout au long de l\\u2019histoire, les religions et les id\\u00e9ologies n\\u2019ont pas\\n                            sanctifi\\u00e9 la vie elle-m\\u00eame, mais autre chose au-dessus ou au-del\\u00e0 de\\n                            l\\u2019existence terrestre. Elles ont donc parfaitement tol\\u00e9r\\u00e9 la mort.\\n                            Certaines ont m\\u00eame montr\\u00e9 beaucoup d\\u2019affection pour la Grande Faucheuse.\\n                            Pour le christianisme, l\\u2019islam et l\\u2019hindouisme, le sens de notre\\n                            existence d\\u00e9pendait de notre destin dans l\\u2019au-del\\u00e0\\u00a0; pour ces\\n                            religions, la mort \\u00e9tait donc un \\u00e9l\\u00e9ment vital et positif du monde. Les\\n                            hommes mouraient parce que Dieu le d\\u00e9cr\\u00e9tait, et l\\u2019heure de la mort\\n                            \\u00e9tait une exp\\u00e9rience m\\u00e9taphysique sacr\\u00e9e gorg\\u00e9e de sens. Quand un homme\\n                            \\u00e9tait sur le point de rendre son dernier soupir, c\\u2019\\u00e9tait le moment\\n                            d\\u2019appeler pr\\u00eatres, rabbins et chamanes pour dresser lebilan de sa vie et embrasser son v\\u00e9ritable r\\u00f4le dans l\\u2019univers.\\n                            Essayez un peu d\\u2019imaginer le christianisme, l\\u2019islam et l\\u2019hindouisme dans\\n                            un monde sans mort, et donc aussi un monde sans ciel, sans enfer ni\\n                            r\\u00e9incarnation.", "cluster": 130}, {"text": "La science et la culture modernes ont une tout autre approche de la vie\\n                            et de la mort. Elles ne voient pas en elle un myst\\u00e8re m\\u00e9taphysique et\\n                            certainement pas la source du sens de la vie. Pour les modernes, la mort\\n                            est plut\\u00f4t un probl\\u00e8me technique que nous pouvons et devons\\n                            r\\u00e9soudre.", "cluster": 130}, {"text": "Comment meurent exactement les hommes\\u00a0? Les contes de f\\u00e9es du Moyen\\n                            \\u00c2ge repr\\u00e9sentaient la mort sous l\\u2019apparence d\\u2019une figure v\\u00eatue d\\u2019un\\n                            manteau noir \\u00e0 capuche, une grande faux \\u00e0 la main. Un homme vit sa vie,\\n                            se tracassant pour ceci ou cela, courant ici ou l\\u00e0, quand soudain para\\u00eet\\n                            devant lui la Grande Faucheuse\\u00a0: elle lui donne une petite tape\\n                            sur l\\u2019\\u00e9paule de l\\u2019un de ses doigts osseux\\u00a0:\\n                            \\u00ab\\u00a0Viens\\u00a0!\\u00a0\\u00bb Et l\\u2019homme implore\\u00a0: \\u00ab\\u00a0S\\u2019il\\n                            vous pla\\u00eet\\u00a0! Encore un an, un mois, une minute\\u00a0!\\u00a0\\u00bb\\n                            Mais la figure encapuchonn\\u00e9e siffle\\u00a0: \\u00ab\\u00a0Non\\u00a0! Tu dois\\n                            venirMAINTENANT!\\u00a0\\u00bb C\\u2019est ainsi que nous\\n                            mourons.", "cluster": 130}, {"text": "5.\\u00a0La mort personnifi\\u00e9e en Grande Faucheuse dans l\\u2019art\\n                                    m\\u00e9di\\u00e9val.", "cluster": 52}, {"text": "En r\\u00e9alit\\u00e9, cependant, les hommes ne meurent pas parce\\n                            qu\\u2019un personnage en manteau noir leur tapote l\\u2019\\u00e9paule, que Dieu l\\u2019a\\n                            d\\u00e9cr\\u00e9t\\u00e9, ou que la mortalit\\u00e9 est une partie essentielle d\\u2019un plus grand\\n                            dessein cosmique. Les humains meurent toujours des suites d\\u2019un p\\u00e9pin\\n                            technique. Le c\\u0153ur cesse de pomper le sang. Des d\\u00e9p\\u00f4ts de graisse\\n                            bouchent l\\u2019art\\u00e8re principale. Des cellules canc\\u00e9reuses se r\\u00e9pandent dans\\n                            le foie. Les germes se multiplient dans les poumons. Et qu\\u2019est-ce qui\\n                            est responsable de tous ces probl\\u00e8mes techniques\\u00a0? D\\u2019autres\\n                            probl\\u00e8mes techniques. Le c\\u0153ur cesse de pomper le sang parce que le\\n                            myocarde ne re\\u00e7oit pas assez d\\u2019oxyg\\u00e8ne. Les cellules canc\\u00e9reuses se\\n                            propagent parce qu\\u2019une mutation g\\u00e9n\\u00e9tique al\\u00e9atoire leur donne de\\n                            nouvelles instructions. Des germes se sont insinu\\u00e9s dans mes poumons\\n                            parce que quelqu\\u2019un a \\u00e9ternu\\u00e9 dans le m\\u00e9tro. Rien de m\\u00e9taphysique dans\\n                            tout cela. Uniquement des probl\\u00e8mes techniques.", "cluster": 130}, {"text": "Et tout probl\\u00e8me technique a une solution technique. Nul n\\u2019est besoin\\n                            d\\u2019attendre le Second Av\\u00e8nement pour triompher de la mort. Deuxgeeks\\u2013\\u00a0ou as de l\\u2019informatique\\u00a0\\u2013 peuvent y parvenir dans\\n                            leur labo. Si, traditionnellement, la mort \\u00e9tait le domaine des pr\\u00eatres\\n                            et des th\\u00e9ologiens, ce sont les ing\\u00e9nieurs qui prennent aujourd\\u2019hui la\\n                            rel\\u00e8ve. Nous pouvons tuer les cellules canc\\u00e9reuses par la chimioth\\u00e9rapie\\n                            ou des nanorobots. Nous pouvons exterminer les germes de nos poumons par\\n                            des antibiotiques. Si le c\\u0153ur s\\u2019arr\\u00eate, nous pouvons le ranimer par des\\n                            m\\u00e9dicaments ou des \\u00e9lectrochocs \\u2013\\u00a0et si \\u00e7a ne marche pas, on peut\\n                            implanter un nouveau c\\u0153ur. Certes, pour l\\u2019heure, nous n\\u2019avons pas de\\n                            solutions \\u00e0 tous les probl\\u00e8mes techniques, mais c\\u2019est pr\\u00e9cis\\u00e9ment pour\\n                            cette raison que nous consacrons tant de temps et d\\u2019argent \\u00e0 la\\n                            recherche sur le cancer, les germes, la g\\u00e9n\\u00e9tique et les\\n                            nanotechnologies.", "cluster": 130}, {"text": "M\\u00eame les gens ordinaires qui ne sont pas impliqu\\u00e9s dans la recherche\\n                            scientifique ont pris l\\u2019habitude de penser \\u00e0 la mort comme \\u00e0 un probl\\u00e8me\\n                            technique. Quand une femme va consulter son m\\u00e9decin et demande\\u00a0:\\n                            \\u00ab\\u00a0Docteur, qu\\u2019est-ce qui ne va pas\\u00a0?\\u00a0\\u00bb, ce dernier\\n                            r\\u00e9pondra probablement\\u00a0: \\u00ab\\u00a0C\\u2019est la grippe\\u00a0\\u00bb,\\n                            \\u00ab\\u00a0C\\u2019est la tuberculose\\u00a0\\u00bb ou encore \\u00ab\\u00a0C\\u2019est un\\n                            cancer\\u00a0\\u00bb. Mais jamais il ne dira\\u00a0: \\u00ab\\u00a0C\\u2019est la\\n                            mort\\u00a0!\\u00a0\\u00bb Et nous vivons tous dans l\\u2019id\\u00e9e que grippe,\\n                            tuberculose et cancer sont des probl\\u00e8mes techniques auxquels nous\\n                            pourrions un jour trouver une solution technique.", "cluster": 130}, {"text": "M\\u00eame quand des gens meurent dans un ouragan, un accident\\n                            de la route ou une guerre, nous avons tendance \\u00e0 y voir un \\u00e9chec\\n                            technique qui aurait pu et d\\u00fb \\u00eatre \\u00e9vit\\u00e9. Si seulement le gouvernement\\n                            avait mis en \\u0153uvre une meilleure politique\\u00a0; si la municipalit\\u00e9\\n                            avait bien fait son travail\\u00a0; si le chef des arm\\u00e9es avait mieux\\n                            r\\u00e9agi, la mort aurait pu \\u00eatre \\u00e9vit\\u00e9e. La mort est devenue une cause\\n                            presque automatique de poursuites et d\\u2019enqu\\u00eates. \\u00ab\\u00a0Comment ont-ils\\n                            pu mourir\\u00a0? Quelqu\\u2019un, quelque part, a failli\\u00a0!\\u00a0\\u00bb", "cluster": -1}, {"text": "L\\u2019immense majorit\\u00e9 des chercheurs, des m\\u00e9decins et des sp\\u00e9cialistes se\\n                            tiennent encore \\u00e0 distance de ces r\\u00eaves affich\\u00e9s d\\u2019immortalit\\u00e9,\\n                            affirmant qu\\u2019ils essaient simplement de surmonter tel ou tel probl\\u00e8me\\n                            particulier. Mais puisque le grand \\u00e2ge et la mort ne sont que le fruit\\n                            de probl\\u00e8mes particuliers, il n\\u2019y a pas de stade auquel m\\u00e9decins et\\n                            chercheurs vont d\\u00e9clarer\\u00a0: \\u00ab\\u00a0C\\u2019est bon, on s\\u2019arr\\u00eate l\\u00e0. Nous\\n                            avons triomph\\u00e9 de la tuberculose et du cancer, mais nous ne l\\u00e8verons pas\\n                            le petit doigt pour combattre la maladie d\\u2019Alzheimer. Que les gens\\n                            continuent \\u00e0 en mourir\\u00a0!\\u00a0\\u00bb La D\\u00e9claration universelle des\\n                            droits de l\\u2019homme ne dit pas que les hommes ont le \\u00ab\\u00a0droit de\\n                            vivre jusqu\\u2019\\u00e0 quatre-vingt-dix ans\\u00a0\\u00bb, mais que tout \\u00eatre humain a\\n                            droit \\u00e0 la vie, point barre. Ce droit n\\u2019est limit\\u00e9 par aucune date\\n                            d\\u2019expiration.", "cluster": 247}, {"text": "Aussi une minorit\\u00e9 croissante de chercheurs et de penseurs parlent-ils\\n                            plus franchement, ces temps-ci, et assurent que le projet phare de la\\n                            science moderne est de vaincre la mort et d\\u2019offrir aux humains\\n                            l\\u2019\\u00e9ternelle jeunesse. Ainsi du g\\u00e9rontologue Aubrey de Grey et du\\n                            polymathe et inventeur Ray Kurzweil (r\\u00e9compens\\u00e9 en 1999 par\\n                            l\\u2019US\\u00a0National Medal of Technology and Innovation). En 2012 Kurzweil a\\n                            \\u00e9t\\u00e9 nomm\\u00e9 directeur de l\\u2019ing\\u00e9nierie chez Google, et un an plus tard\\n                            Google a lanc\\u00e9 une filiale, Calico, dont la mission d\\u00e9clar\\u00e9e est de\\n                            \\u00ab\\u00a0r\\u00e9soudre le probl\\u00e8me de la mort(27)\\u00bb.\\n                            Google a derni\\u00e8rement nomm\\u00e9 un autre convaincu de l\\u2019immortalit\\u00e9, Bill\\n                            Maris, \\u00e0 la t\\u00eate du fonds d\\u2019investissement Google Ventures. Dans une\\n                            interview de janvier 2015, Maris d\\u00e9clarait\\u00a0: \\u00ab\\u00a0Si vous me\\n                            demandez aujourd\\u2019hui s\\u2019il est possible de vivre jusqu\\u2019\\u00e0 cinq cents\\u00a0ans,\\n                            la r\\u00e9ponse est oui.\\u00a0\\u00bb Maris \\u00e9taye ses propos audacieux par une\\n                            forte somme d\\u2019argent liquide\\u00a0: Google Ventures investit 36\\u00a0% de\\n                            ses deux\\u00a0milliards de dollars en portefeuille dans des start-up\\n                            sp\\u00e9cialis\\u00e9es en sciences de la vie, dont plusieurs projets ambitieux\\n                            visant\\u00e0 prolonger la vie. Recourant \\u00e0 une analogie avec\\n                            le football am\\u00e9ricain, Maris ajoute\\u00a0: dans le combat contre la\\n                            mort, \\u00ab\\u00a0nous n\\u2019essayons pas de gagner quelques m\\u00e8tres. Nous\\n                            cherchons \\u00e0 gagner la partie\\u00a0\\u00bb. Pourquoi\\u00a0? \\u00ab\\u00a0Parce\\n                            que, r\\u00e9pond-il, mieux vaut vivre que mourir(28).\\u00a0\\u00bb", "cluster": 273}, {"text": "Ces r\\u00eaves sont partag\\u00e9s par d\\u2019autres sommit\\u00e9s de la Silicon Valley. Le\\n                            cofondateur de PayPal, Peter Thiel, a derni\\u00e8rement confess\\u00e9 qu\\u2019il compte\\n                            bien vivre \\u00e9ternellement. \\u00ab\\u00a0Je pense qu\\u2019il y a probablement trois\\n                            grandes fa\\u00e7ons d\\u2019aborder [la mort], a-t-il expliqu\\u00e9. L\\u2019accepter, la nier\\n                            ou la combattre. Je crois que notre soci\\u00e9t\\u00e9 est domin\\u00e9e par des gens qui\\n                            sont dans le d\\u00e9ni ou l\\u2019acceptation\\u00a0; pour ma part, je pr\\u00e9f\\u00e8re la\\n                            combattre.\\u00a0\\u00bb Beaucoup de gens balaieront probablement ces\\n                            d\\u00e9clarations d\\u2019un revers de main, y voyant des fantaisies d\\u2019adolescents.\\n                            Or, Thiel est quelqu\\u2019un qu\\u2019il faut prendre tr\\u00e8s au s\\u00e9rieux. Il est un\\n                            des entrepreneurs les plus prosp\\u00e8res et influents de la Silicon Valley,\\n                            \\u00e0 la t\\u00eate d\\u2019une fortune priv\\u00e9e estim\\u00e9e \\u00e0 2,2\\u00a0milliards de dollars(29). La messe est dite\\u00a0: l\\u2019\\u00e9galit\\u00e9 est hors jeu,\\n                            l\\u2019immortalit\\u00e9 dans l\\u2019air du temps.", "cluster": 273}, {"text": "Le d\\u00e9veloppement \\u00e0 vitesse grand V de domaines comme le g\\u00e9nie g\\u00e9n\\u00e9tique,\\n                            la m\\u00e9decine r\\u00e9g\\u00e9n\\u00e9rative et les nanotechnologies nourrit des proph\\u00e9ties\\n                            toujours plus optimistes. Certains experts croient que les humains\\n                            triompheront de la mort d\\u2019ici 2200, d\\u2019autres parlent m\\u00eame de 2100.\\n                            Kurzweil et de\\u00a0Grey sont encore plus confiants. Ils soutiennent qu\\u2019en\\n                            2050 quiconque poss\\u00e8de un corps sain et un solide compte en banque aura\\n                            une chance s\\u00e9rieuse d\\u2019acc\\u00e9der \\u00e0 l\\u2019immortalit\\u00e9 en trompant la mort de\\n                            d\\u00e9cennie en d\\u00e9cennie. Tous les dix ans, selon Kurzweil et de\\u00a0Grey, nous\\n                            ferons un s\\u00e9jour dans une clinique pour y subir une transformation qui\\n                            nous gu\\u00e9rira de nos maladies, mais r\\u00e9g\\u00e9n\\u00e9rera aussi nos tissus en\\n                            d\\u00e9composition et am\\u00e9liorera nos mains, nos yeux et notre cerveau. Entre\\n                            deux hospitalisations, les m\\u00e9decins auront invent\\u00e9 pl\\u00e9thore de nouveaux\\n                            m\\u00e9dicaments, d\\u2019extensions et de gadgets. Si Kurzweil et de\\u00a0Grey ont\\n                            raison, peut-\\u00eatre y a-t-il d\\u00e9j\\u00e0 des immortels qui marchent \\u00e0 c\\u00f4t\\u00e9 de\\n                            vous dans la rue \\u2013\\u00a0du moins si vous arpentez Wall Street ou la Cinqui\\u00e8me\\n                            Avenue.", "cluster": 273}, {"text": "En v\\u00e9rit\\u00e9, ils seront a-mortels plut\\u00f4t qu\\u2019immortels. \\u00c0 la diff\\u00e9rence de\\n                            Dieu, les surhommes futurs pourraient encore mourir dans une guerre ou\\n                            un accident, et rien ne pourrait les faire revenir des enfers.\\n                            Toutefois, contrairement \\u00e0 nous autres, mortels, leur vie n\\u2019aurait pas\\n                            dedate d\\u2019expiration. Tant qu\\u2019une bombe ne les r\\u00e9duirait\\n                            pas en charpie ou qu\\u2019ils ne passeraient pas sous un camion, ils\\n                            pourraient vivre ind\\u00e9finiment. Ce qui ferait probablement d\\u2019eux les gens\\n                            les plus angoiss\\u00e9s de l\\u2019histoire. Nous autres, mortels, prenons chaque\\n                            jour des risques avec notre existence, car nous savons qu\\u2019elle aura une\\n                            fin de toute fa\\u00e7on. Nous partons donc en randonn\\u00e9e dans l\\u2019Himalaya,\\n                            nageons dans la mer et faisons quantit\\u00e9 d\\u2019autres choses dangereuses,\\n                            comme traverser la rue ou aller au restaurant. Mais devant la\\n                            perspective de vivre \\u00e9ternellement, qui serait assez fou pour prendre\\n                            des paris avec l\\u2019infini comme \\u00e7a\\u00a0?", "cluster": -1}, {"text": "Mieux vaudrait peut-\\u00eatre alors commencer par des objectifs plus modestes,\\n                            comme multiplier par deux l\\u2019esp\\u00e9rance de vie\\u00a0? AuXXesi\\u00e8cle, celle-ci a presque doubl\\u00e9 pour\\n                            passer de quarante \\u00e0 soixante-dix ans\\u00a0; auXXIesi\\u00e8cle, nous devrions au moins pouvoir la\\n                            doubler encore et atteindre cent cinquante ans. Nous serions toujours\\n                            tr\\u00e8s loin de l\\u2019immortalit\\u00e9, mais cela r\\u00e9volutionnerait encore la\\n                            soci\\u00e9t\\u00e9. Pour commencer, la structure familiale, les mariages et les\\n                            relations parent-enfant s\\u2019en trouveraient transform\\u00e9s. Aujourd\\u2019hui, les\\n                            gens s\\u2019imaginent encore mari\\u00e9s \\u00ab\\u00a0jusqu\\u2019\\u00e0 ce que la mort les\\n                            s\\u00e9pare\\u00a0\\u00bb, et une bonne partie de leur vie tourne autour de\\n                            l\\u2019\\u00e9ducation des enfants. Essayons maintenant d\\u2019imaginer une personne\\n                            dont la dur\\u00e9e de vie est de cent cinquante ans. \\u00c0 supposer qu\\u2019elle se\\n                            marie \\u00e0 quarante ans, il lui reste cent dix ans \\u00e0 vivre. Sera-t-il\\n                            r\\u00e9aliste d\\u2019esp\\u00e9rer que son couple dure cent dix ans\\u00a0? Il y a de\\n                            quoi faire h\\u00e9siter m\\u00eame les catholiques int\\u00e9gristes. La tendance\\n                            actuelle aux mariages en s\\u00e9rie est donc susceptible de s\\u2019intensifier. \\u00c0\\n                            cent vingt ans, une femme qui aura eu deux enfants dans la quarantaine\\n                            n\\u2019aura qu\\u2019un lointain souvenir des ann\\u00e9es pass\\u00e9es \\u00e0 les \\u00e9lever, qui\\n                            seront comme un \\u00e9pisode plut\\u00f4t mineur de sa longue vie. Il est difficile\\n                            de dire quel genre de relations parent-enfant pourrait se d\\u00e9velopper\\n                            dans ces circonstances.", "cluster": 163}, {"text": "Ou consid\\u00e9rons les carri\\u00e8res professionnelles. Aujourd\\u2019hui, on est cens\\u00e9\\n                            faire l\\u2019apprentissage de son m\\u00e9tier dans l\\u2019adolescence ou au d\\u00e9but de\\n                            l\\u2019\\u00e2ge adulte, puis passer le reste de sa vie dans le m\\u00eame secteur. De\\n                            toute \\u00e9vidence, on apprend encore dans la quarantaine ou la\\n                            cinquantaine, mais la vie est g\\u00e9n\\u00e9ralement divis\\u00e9e en une p\\u00e9riode\\n                            d\\u2019apprentissage suivie d\\u2019une p\\u00e9riode de travail. Quand vous vivrezjusqu\\u2019\\u00e0 cent cinquante ans, il n\\u2019en ira plus ainsi,\\n                            surtout dans un monde constamment \\u00e9branl\\u00e9 par de nouvelles technologies.\\n                            Les gens auront des carri\\u00e8res beaucoup plus longues et devront se\\n                            r\\u00e9inventer sans cesse, f\\u00fbt-ce \\u00e0 quatre-vingt-dix ans.", "cluster": 107}, {"text": "Dans le m\\u00eame temps, les gens ne prendront pas leur retraite \\u00e0\\n                            soixante-cinq ans et ne laisseront pas la place \\u00e0 la nouvelle\\n                            g\\u00e9n\\u00e9ration, avec ses id\\u00e9es neuves et ses aspirations. On conna\\u00eet le mot\\n                            c\\u00e9l\\u00e8bre de Max Planck\\u00a0: la science n\\u2019avance que d\\u2019un enterrement \\u00e0\\n                            la fois. Il entendait par l\\u00e0 que les th\\u00e9ories nouvelles n\\u2019ont une chance\\n                            de d\\u00e9raciner les anciennes qu\\u2019\\u00e0 la disparition d\\u2019une g\\u00e9n\\u00e9ration. Ce\\n                            n\\u2019est pas seulement vrai de la science. Pensez un instant \\u00e0 votre lieu\\n                            de travail. Peu importe que vous soyez chercheur, journaliste, cuisinier\\n                            ou footballeur\\u00a0: qu\\u2019\\u00e9prouveriez-vous \\u00e0 avoir un patron de cent\\n                            vingt ans, dont les id\\u00e9es ont \\u00e9t\\u00e9 formul\\u00e9es du temps de la reine\\n                            Victoria et qui aurait toute chance de rester votre chef encore une\\n                            vingtaine d\\u2019ann\\u00e9es\\u00a0?", "cluster": 107}, {"text": "Dans la sph\\u00e8re politique, les r\\u00e9sultats pourraient \\u00eatre encore plus\\n                            d\\u00e9sastreux. Seriez-vous contents d\\u2019avoir Poutine encore quatre-vingt-dix\\n                            ans\\u00a0? \\u00c0 la r\\u00e9flexion, si l\\u2019on vivait jusqu\\u2019\\u00e0 cent cinquante ans,\\n                            Staline serait encore au pouvoir \\u00e0 Moscou, dans la force de l\\u2019\\u00e2ge \\u00e0 cent\\n                            trente-huit ans\\u00a0; \\u00e0 cent vingt-huit ans, le pr\\u00e9sident Mao serait\\n                            dans l\\u2019\\u00e2ge m\\u00fbr, et la princesse Elisabeth se tournerait les pouces en\\n                            attendant d\\u2019h\\u00e9riter d\\u2019un George\\u00a0VI de cent vingt et un ans, tandis que\\n                            son fils, le prince Charles, devrait attendre son tour jusqu\\u2019en\\n                            2076.", "cluster": -1}, {"text": "Revenons \\u00e0 la r\\u00e9alit\\u00e9\\u00a0: il est loin d\\u2019\\u00eatre certain que les\\n                            proph\\u00e9ties de Kurzweil et de\\u00a0Grey se r\\u00e9alisent d\\u2019ici 2050 ou 2100. \\u00c0 mon\\n                            sens, esp\\u00e9rer parvenir \\u00e0 l\\u2019\\u00e9ternelle jeunesse auXXIesi\\u00e8cle est pr\\u00e9matur\\u00e9, et qui les prend au\\n                            s\\u00e9rieux est vou\\u00e9 \\u00e0 une cruelle d\\u00e9ception. Il n\\u2019est pas facile de vivre\\n                            en sachant que vous allez mourir, mais il est encore plus dur de croire\\n                            \\u00e0 l\\u2019immortalit\\u00e9 et de se tromper.", "cluster": 273}, {"text": "Si l\\u2019esp\\u00e9rance de vie moyenne a doubl\\u00e9 au cours des cent derni\\u00e8res\\n                            ann\\u00e9es, rien ne justifie d\\u2019extrapoler et d\\u2019en conclure que nous pouvons\\n                            de nouveau la doubler, pour la porter \\u00e0 cent cinquante ans au cours du\\n                            si\\u00e8cle prochain. En 1900, l\\u2019esp\\u00e9rance de vie dans le monde ne d\\u00e9passait\\n                            pas quarante ans car beaucoup de gens mouraient jeunes de malnutrition,\\n                            de maladies infectieuses ou de violences. En revanche, ceux qui\\n                            \\u00e9chappaient \\u00e0 la famine, \\u00e0 la peste ou \\u00e0 la guerre pouvaient vivrejusqu\\u2019\\u00e0 soixante-dix ou quatre-vingts ans pass\\u00e9s, qui\\n                            est la dur\\u00e9e de vie naturelle d\\u2019Homo sapiens. Contrairement \\u00e0 ce\\n                            que l\\u2019on croit, les gens de soixante-dix ans ne passaient pas pour des\\n                            b\\u00eates curieuses dans les si\\u00e8cles pass\\u00e9s. Galil\\u00e9e est mort \\u00e0\\n                            soixante-dix-sept ans, Isaac Newton \\u00e0 quatre-vingt-quatre, et\\n                            Michel-Ange a v\\u00e9cu jusqu\\u2019\\u00e0 l\\u2019\\u00e2ge avanc\\u00e9 de quatre-vingt-huit ans sans\\n                            antibiotiques, vaccins ou transplantations d\\u2019organes. De fait, m\\u00eame les\\n                            chimpanz\\u00e9s dans la jungle vivent parfois jusqu\\u2019\\u00e0 plus de soixante\\n                                ans(30).", "cluster": 163}, {"text": "En v\\u00e9rit\\u00e9, jusqu\\u2019ici, la m\\u00e9decine moderne n\\u2019a pas augment\\u00e9 notre dur\\u00e9e de\\n                            vie naturelle d\\u2019une seule ann\\u00e9e. Sa grande prouesse a \\u00e9t\\u00e9 de nous sauver\\n                            d\\u2019une mortpr\\u00e9matur\\u00e9eet de nous permettre de jouir pleinement du\\n                            nombre de nos ann\\u00e9es. Quand bien m\\u00eame nous triompherions du cancer, du\\n                            diab\\u00e8te et des autres grandes maladies mortelles, cela signifierait\\n                            simplement que tout le monde, ou presque, vivrait jusqu\\u2019\\u00e0\\n                            quatre-vingt-dix ans\\u00a0: cela ne suffirait pas pour atteindre cent\\n                            cinquante ans, sans parler de cinq cents ans. Pour ce faire, la m\\u00e9decine\\n                            devra r\\u00e9agencer les structures et processus les plus fondamentaux du\\n                            corps humain, et d\\u00e9couvrir comment r\\u00e9g\\u00e9n\\u00e9rer les organes et les tissus.\\n                            Que nous puissions le faire en 2100 est loin d\\u2019\\u00eatre \\u00e9vident.", "cluster": 163}, {"text": "Chaque tentative rat\\u00e9e de triompher de la mort nous rapprochera n\\u00e9anmoins\\n                            un peu plus de ce but, nourrira de plus grands espoirs et encouragera\\n                            les gens \\u00e0 consentir de plus grands efforts. Calico ne r\\u00e9soudra\\n                            vraisemblablement pas le probl\\u00e8me de la mort \\u00e0 temps pour rendre\\n                            immortels les cofondateurs de Google, Sergey Brin et Larry Page, mais\\n                            elle r\\u00e9alisera tr\\u00e8s probablement des d\\u00e9couvertes significatives en\\n                            mati\\u00e8re de biologie cellulaire, de m\\u00e9dicaments g\\u00e9n\\u00e9tiques et de sant\\u00e9\\n                            humaine. La prochaine g\\u00e9n\\u00e9ration de Googlers pourrait donc bien lancer\\n                            son offensive contre la mort en partant de nouvelles bases plus solides.\\n                            Les hommes de science qui crient \\u00e0 l\\u2019immortalit\\u00e9 sont comme l\\u2019enfant qui\\n                            criait au loup\\u00a0: t\\u00f4t ou tard, le loup finit par venir.", "cluster": 273}, {"text": "D\\u00e8s lors, m\\u00eame si nous n\\u2019acc\\u00e9dons pas \\u00e0 l\\u2019immortalit\\u00e9 de notre vivant, la\\n                            guerre contre la mort restera probablement le projet phare du si\\u00e8cle\\n                            prochain. Si vous faites entrer en ligne de compte notre croyance en la\\n                            saintet\\u00e9 de la vie humaine, que vous y ajoutiez la dynamique de\\n                                l\\u2019establishmentscientifique et couronniez le tout par les\\n                            besoins de l\\u2019\\u00e9conomie capitaliste, une guerre implacable contre la mort\\n                            para\\u00eetin\\u00e9vitable. Notre attachement id\\u00e9ologique \\u00e0 la\\n                            vie ne nous permettra jamais d\\u2019accepter simplement la mort humaine. Tant\\n                            que des gens mourront de quelque chose, nous nous efforcerons de\\n                            l\\u2019emp\\u00eacher.", "cluster": 247}, {"text": "L\\u2019establishmentscientifique et l\\u2019\\u00e9conomie capitaliste seront plus\\n                            qu\\u2019heureux d\\u2019\\u00e9pauler ce combat. La plupart des hommes de science et des\\n                            banquiers se fichent pas mal de ce sur quoi ils travaillent, du moment\\n                            que c\\u2019est l\\u2019occasion de nouvelles d\\u00e9couvertes et de plus gros profits.\\n                            Peut-on imaginer d\\u00e9fi scientifique plus excitant que de duper la mort,\\n                            ou march\\u00e9 plus prometteur que celui de la jeunesse \\u00e9ternelle\\u00a0?\\n                            Vous avez plus de quarante ans\\u00a0? Fermez les yeux une minute et\\n                            t\\u00e2chez de vous rappeler le corps qui \\u00e9tait le v\\u00f4tre \\u00e0 vingt-cinq ans.\\n                            Pas seulement son apparence, mais les sensations que vous \\u00e9prouviez. Si\\n                            vous pouviez retrouver ce corps, combien consentiriez-vous \\u00e0\\n                            payer\\u00a0? D\\u2019aucuns, sans doute, y renonceraient volontiers, mais\\n                            suffisamment de clients seraient pr\\u00eats \\u00e0 faire le n\\u00e9cessaire pour que se\\n                            cr\\u00e9e un march\\u00e9 presque infini.", "cluster": 248}, {"text": "Si tout cela ne suffit pas, la peur de la mort enracin\\u00e9e chez la plupart\\n                            des \\u00eatres humains donnera \\u00e0 la guerre contre la mort un souffle\\n                            irr\\u00e9sistible. Aussi longtemps que les gens ont cru la mort in\\u00e9vitable,\\n                            ils se sont exerc\\u00e9s assez t\\u00f4t \\u00e0 refouler le d\\u00e9sir de vivre \\u00e0 jamais, ou\\n                            l\\u2019ont attel\\u00e9 \\u00e0 des objectifs de substitution. Les gens veulent vivre \\u00e0\\n                            jamais, alors ils composent une symphonie\\n                            \\u00ab\\u00a0immortelle\\u00a0\\u00bb\\u00a0; ils recherchent la \\u00ab\\u00a0gloire\\n                            \\u00e9ternelle\\u00a0\\u00bb dans une guerre, voire sacrifient leur vie pour que\\n                            leur \\u00e2me \\u00ab\\u00a0jouisse de la f\\u00e9licit\\u00e9 \\u00e9ternelle au paradis\\u00a0\\u00bb. La\\n                            peur de la mort alimente largement notre cr\\u00e9ativit\\u00e9 artistique, notre\\n                            engagement politique et notre pi\\u00e9t\\u00e9 religieuse.", "cluster": 247}, {"text": "Anim\\u00e9 par la peur de la mort, Woody Allen a fait une carri\\u00e8re fabuleuse.\\n                            Alors qu\\u2019on lui demandait un jour s\\u2019il esp\\u00e9rait vivre \\u00e9ternellement \\u00e0\\n                            l\\u2019\\u00e9cran, il a r\\u00e9pondu\\u00a0: \\u00ab\\u00a0Je pr\\u00e9f\\u00e9rerais vivre dans mon\\n                            appartement.\\u00a0\\u00bb Et d\\u2019ajouter aussit\\u00f4t\\u00a0: \\u00ab\\u00a0Je ne veux\\n                            pas acc\\u00e9der \\u00e0 l\\u2019immortalit\\u00e9 par mon travail. Je veux l\\u2019obtenir en ne\\n                            mourant pas.\\u00a0\\u00bb La gloire \\u00e9ternelle, les comm\\u00e9morations\\n                            nationalistes ou les r\\u00eaves de paradis sont de pi\\u00e8tres substituts de ce\\n                            que d\\u00e9sirent les hommes comme Woody Allen\\u00a0: ne pas mourir. Le jour\\n                            o\\u00f9 les gens penseront (\\u00e0 tort ou \\u00e0 raison) qu\\u2019ils ont une chance\\n                            s\\u00e9rieuse d\\u2019\\u00e9chapper \\u00e0 la mort, le d\\u00e9sir de vivre refusera de continuer \\u00e0\\n                            tirer le wagon bringuebalant de l\\u2019art, de l\\u2019id\\u00e9ologie et de la religion,\\n                            et emportera tout comme une avalanche.", "cluster": -1}, {"text": "Vous trouvez impitoyables les fanatiques religieux au\\n                            regard br\\u00fblant et \\u00e0 la barbe fleurie\\u00a0? Attendez un peu de voir ce\\n                            que feront les vieux nababs entrepreneurs et les starlettes\\n                            vieillissantes d\\u2019Hollywood s\\u2019ils pensent qu\\u2019un \\u00e9lixir de vie est \\u00e0\\n                            port\\u00e9e de main. Le jour o\\u00f9 la science accomplira des progr\\u00e8s\\n                            significatifs dans la guerre contre la mort, la vraie bataille se\\n                            d\\u00e9placera des laboratoires vers les parlements, les tribunaux et la rue.\\n                            D\\u00e8s que les efforts scientifiques seront couronn\\u00e9s de succ\\u00e8s, ils\\n                            d\\u00e9clencheront d\\u2019\\u00e2pres conflits politiques. Les guerres et les conflits\\n                            qui ont jalonn\\u00e9 l\\u2019histoire pourraient bien n\\u2019\\u00eatre qu\\u2019un p\\u00e2le pr\\u00e9lude au\\n                            vrai combat qui nous attend\\u00a0: le combat pour la jeunesse\\n                            \\u00e9ternelle.", "cluster": 248}, {"text": "Le second grand projet \\u00e0 l\\u2019ordre du jour de l\\u2019humanit\\u00e9 sera probablement\\n                            de trouver la cl\\u00e9 du bonheur. Tout au long de l\\u2019histoire, nombreux sont\\n                            les penseurs, proph\\u00e8tes et gens ordinaires qui ont fait du bonheur,\\n                            plut\\u00f4t que de la vie, le bien supr\\u00eame. Dans la Gr\\u00e8ce antique, \\u00c9picure\\n                            expliquait qu\\u2019adorer les dieux est une perte de temps, qu\\u2019il n\\u2019y a pas\\n                            de vie apr\\u00e8s la mort et que le bonheur est l\\u2019unique but de la vie. Dans\\n                            l\\u2019Antiquit\\u00e9, la plupart des citoyens rejetaient l\\u2019\\u00e9picurisme, mais\\n                            aujourd\\u2019hui il est devenu le point de vue par d\\u00e9faut. Le scepticisme \\u00e0\\n                            propos de l\\u2019au-del\\u00e0 pousse l\\u2019humanit\\u00e9 \\u00e0 chercher non seulement\\n                            l\\u2019immortalit\\u00e9, mais aussi le bonheur terrestre. Car qui voudrait vivre\\n                            dans une mis\\u00e8re \\u00e9ternelle\\u00a0?", "cluster": -1}, {"text": "Pour \\u00c9picure, la poursuite du bonheur \\u00e9tait une qu\\u00eate personnelle. Les\\n                            penseurs modernes, en revanche, ont tendance \\u00e0 y voir un projet\\n                            collectif. Sans planification de l\\u2019\\u00c9tat, sans ressources \\u00e9conomiques ni\\n                            recherche scientifique, les individus n\\u2019iront pas bien loin dans leur\\n                            qu\\u00eate du bonheur. Si votre pays est d\\u00e9chir\\u00e9 par la guerre, l\\u2019\\u00e9conomie en\\n                            crise et le syst\\u00e8me de sant\\u00e9 inexistant, vous vivrez mis\\u00e9rablement. Le\\n                            bien supr\\u00eame est \\u00ab\\u00a0le plus grand bonheur du plus grand\\n                            nombre\\u00a0\\u00bb, d\\u00e9clara \\u00e0 la fin duXVIIIesi\\u00e8cle le\\n                            philosophe britannique Jeremy Bentham, avant de conclure qu\\u2019accro\\u00eetre le\\n                            bonheur g\\u00e9n\\u00e9ral \\u00e9tait l\\u2019unique objectif digne de l\\u2019\\u00c9tat, du march\\u00e9 et de\\n                            la communaut\\u00e9 scientifique. Les hommespolitiques\\n                            devraient \\u0153uvrer pour la paix, les hommes d\\u2019affaires encourager la\\n                            prosp\\u00e9rit\\u00e9 et les savants \\u00e9tudier la nature, non pas pour la plus grande\\n                            gloire du roi, du pays ou de Dieu, mais pour que vous et moi puissions\\n                            jouir d\\u2019une vie plus heureuse.", "cluster": 222}, {"text": "AuxXIXeetXXesi\\u00e8cles, beaucoup\\n                            ont rendu un hommage de pure forme \\u00e0 la vision de Bentham, mais les\\n                            gouvernements, les soci\\u00e9t\\u00e9s et les laboratoires se sont focalis\\u00e9s sur\\n                            des objectifs plus imm\\u00e9diats et mieux d\\u00e9finis. Les pays ont mesur\\u00e9 leur\\n                            r\\u00e9ussite \\u00e0 la taille de leur territoire, \\u00e0 l\\u2019augmentation de leur\\n                            population, \\u00e0 la croissance de leur PIB, non pas au bonheur de leurs\\n                            citoyens. Des pays industrialis\\u00e9s comme l\\u2019Allemagne, la France et le\\n                            Japon ont mis en place de vastes syst\\u00e8mes d\\u2019\\u00e9ducation, de sant\\u00e9 et de\\n                            protection sociale, mais ceux-ci avaient pour vocation de renforcer la\\n                            nation plut\\u00f4t que d\\u2019assurer le bien-\\u00eatre individuel.", "cluster": 261}, {"text": "On fondait des \\u00e9coles pour produire des citoyens dou\\u00e9s et ob\\u00e9issants qui\\n                            serviraient loyalement la nation. \\u00c0 dix-huit ans, les jeunes devaient\\n                            \\u00eatre non seulement patriotes, mais aussi alphab\\u00e9tis\\u00e9s, de mani\\u00e8re \\u00e0\\n                            pouvoir lire les consignes du brigadier et dresser les plans de bataille\\n                            du lendemain. Ils devaient conna\\u00eetre les math\\u00e9matiques pour calculer la\\n                            trajectoire d\\u2019un obus ou d\\u00e9chiffrer le code secret utilis\\u00e9 par l\\u2019ennemi.\\n                            Ils avaient besoin de ma\\u00eetriser suffisamment l\\u2019\\u00e9lectricit\\u00e9, la m\\u00e9canique\\n                            et la m\\u00e9decine pour se servir de la TSF, conduire des chars et venir en\\n                            aide \\u00e0 leurs camarades bless\\u00e9s. Une fois sortis de l\\u2019arm\\u00e9e, on attendait\\n                            d\\u2019eux qu\\u2019ils servent la nation comme employ\\u00e9s de bureau, enseignants et\\n                            ing\\u00e9nieurs, pour b\\u00e2tir une \\u00e9conomie moderne et payer beaucoup\\n                            d\\u2019imp\\u00f4ts.", "cluster": -1}, {"text": "Il en allait de m\\u00eame pour le syst\\u00e8me de sant\\u00e9. \\u00c0 la fin duXIXesi\\u00e8cle, des pays comme la France,\\n                            l\\u2019Allemagne et le Japon commenc\\u00e8rent \\u00e0 garantir des soins gratuits aux\\n                            masses. Ils financ\\u00e8rent la vaccination des b\\u00e9b\\u00e9s, des r\\u00e9gimes \\u00e9quilibr\\u00e9s\\n                            pour les enfants et l\\u2019\\u00e9ducation physique des adolescents. Ils\\n                            ass\\u00e9ch\\u00e8rent les marais insalubres, extermin\\u00e8rent les moustiques et\\n                            construisirent des r\\u00e9seaux de tout-\\u00e0-l\\u2019\\u00e9gout. L\\u2019objectif n\\u2019\\u00e9tait pas de\\n                            rendre les gens heureux, mais la nation plus forte. Le pays avait besoin\\n                            de soldats et d\\u2019ouvriers robustes, de femmes en bonne sant\\u00e9 qui\\n                            donneraient naissance \\u00e0 d\\u2019autres soldats et ouvriers, et de\\n                            fonctionnaires qui seraient au bureau \\u00e0 8\\u00a0heures du matin pr\\u00e9cises au\\n                            lieu de rester alit\\u00e9s, malades, chez eux.", "cluster": 261}, {"text": "M\\u00eame le syst\\u00e8me de protection sociale a \\u00e9t\\u00e9 initialement\\n                            organis\\u00e9 dans l\\u2019int\\u00e9r\\u00eat de la nation plut\\u00f4t que dans celui des individus\\n                            n\\u00e9cessiteux. \\u00c0 la fin duXIXesi\\u00e8cle, quand Bismarck\\n                            se fit le pionnier des pensions de l\\u2019\\u00c9tat et de la s\\u00e9curit\\u00e9 sociale, son\\n                            principal objectif \\u00e9tait de s\\u2019assurer la loyaut\\u00e9 des citoyens, plut\\u00f4t\\n                            que d\\u2019am\\u00e9liorer leur bien-\\u00eatre. On se battait pour son pays \\u00e0 dix-huit\\n                            ans, et on payait des imp\\u00f4ts \\u00e0 quarante ans parce qu\\u2019on comptait sur\\n                            l\\u2019\\u00c9tat pour s\\u2019occuper de nous \\u00e0 soixante-dix ans(31).", "cluster": -1}, {"text": "En 1776, les P\\u00e8res fondateurs des \\u00c9tats-Unis firent du droit \\u00e0 la\\n                            poursuite du bonheur un des trois droits inali\\u00e9nables de l\\u2019homme, avec\\n                            le droit \\u00e0 la vie et le droit \\u00e0 la libert\\u00e9. Il importe cependant de\\n                            noter que la D\\u00e9claration d\\u2019ind\\u00e9pendance am\\u00e9ricaine garantit le droit \\u00e0la poursuitedu bonheur, non pas le droit au bonheur\\n                            lui-m\\u00eame. Surtout, Thomas Jefferson ne rendit pas l\\u2019\\u00c9tat responsable du\\n                            bonheur de ses citoyens. Il chercha plut\\u00f4t \\u00e0 limiter le pouvoir de\\n                            l\\u2019\\u00c9tat. L\\u2019id\\u00e9e \\u00e9tait de r\\u00e9server aux individus une sph\\u00e8re de choix\\n                            priv\\u00e9e, \\u00e9chappant \\u00e0 la supervision de l\\u2019\\u00c9tat. Si je crois que je serai\\n                            plus heureux en \\u00e9pousant Jean plut\\u00f4t que Marie, en habitant San\\n                            Francisco plut\\u00f4t que Salt Lake City, en \\u00e9tant barman plut\\u00f4t que laitier,\\n                            j\\u2019ai le droit de chercher mon bonheur ainsi, et l\\u2019\\u00c9tat ne doit pas\\n                            intervenir m\\u00eame si je fais le mauvais choix.", "cluster": -1}, {"text": "Au cours des toutes derni\\u00e8res d\\u00e9cennies, cependant, la situation s\\u2019est\\n                            renvers\\u00e9e. La vision de Bentham a \\u00e9t\\u00e9 prise bien plus au s\\u00e9rieux. Les\\n                            gens croient de plus en plus que les immenses syst\\u00e8mes mis en place\\n                            voici plus d\\u2019un si\\u00e8cle pour renforcer la nation doivent r\\u00e9ellement\\n                            servir le bonheur et le bien-\\u00eatre des individus. Nous ne sommes pas ici\\n                            pour servir l\\u2019\\u00c9tat. C\\u2019est \\u00e0 lui de nous servir. Le droit \\u00e0 la poursuite\\n                            du bonheur, initialement envisag\\u00e9 comme un frein \\u00e0 la puissance de\\n                            l\\u2019\\u00c9tat, s\\u2019est imperceptiblement m\\u00e9tamorphos\\u00e9 en droit au bonheur, comme\\n                            si les \\u00eatres humains avaient le droit naturel d\\u2019\\u00eatre heureux, et comme\\n                            si tout ce qui nous laisse insatisfait \\u00e9tait une violation de nos droits\\n                            humains fondamentaux, obligeant l\\u2019\\u00c9tat \\u00e0 intervenir.", "cluster": -1}, {"text": "AuXXesi\\u00e8cle, le PIB par t\\u00eate a peut-\\u00eatre \\u00e9t\\u00e9 l\\u2019aune\\n                            supr\\u00eame \\u00e0 laquelle \\u00e9valuer la r\\u00e9ussite d\\u2019une nation. Dans cette\\n                            perspective, Singapour, dont chaque citoyen produit en moyenne\\n                            56\\u00a0000\\u00a0dollars de biens et services par an, r\\u00e9ussit mieux que le Costa\\n                            Rica, dont les citoyens ne produisent que 14\\u00a0000\\u00a0dollars par an. De nos\\n                            jours, cependant, despenseurs, des hommes politiques et\\n                            m\\u00eame des \\u00e9conomistes appellent \\u00e0 remplacer le PIB par le BIB, le Bonheur\\n                            int\\u00e9rieur brut. Apr\\u00e8s tout, que demande le peuple\\u00a0? Les gens ne\\n                            veulent pas produire. Ils veulent \\u00eatre heureux. La production est\\n                            importante parce qu\\u2019elle assure la base mat\\u00e9rielle du bonheur. Mais elle\\n                            est un moyen, non une fin. Toutes les enqu\\u00eates le prouvent, les\\n                            Costaricains ont des niveaux de satisfaction bien sup\\u00e9rieurs \\u00e0 ceux des\\n                            Singapouriens. Pr\\u00e9f\\u00e9reriez-vous \\u00eatre un Singapourien tr\\u00e8s productif mais\\n                            insatisfait, ou un Costaricain moins productif mais\\n                            satisfait\\u00a0?", "cluster": 15}, {"text": "Ce genre de logique pourrait bien pousser l\\u2019humanit\\u00e9 \\u00e0 faire du bonheur\\n                            son deuxi\\u00e8me grand but auXXIesi\\u00e8cle. \\u00c0 premi\\u00e8re\\n                            vue, ce pourrait \\u00eatre un projet relativement ais\\u00e9 \\u00e0 accomplir. Si la\\n                            famine, la peste et la guerre disparaissent, si l\\u2019esp\\u00e8ce humaine conna\\u00eet\\n                            une paix et une prosp\\u00e9rit\\u00e9 sans pr\\u00e9c\\u00e9dent, et si l\\u2019esp\\u00e9rance de vie\\n                            augmente de mani\\u00e8re spectaculaire, tout cela ne manquera pas de rendre\\n                            les \\u00eatres humains heureux, n\\u2019est-ce pas\\u00a0?", "cluster": -1}, {"text": "Faux. Quand \\u00c9picure d\\u00e9finissait le bonheur comme le bien supr\\u00eame, il\\n                            pr\\u00e9venait ses disciples qu\\u2019il n\\u2019est pas facile d\\u2019\\u00eatre heureux. Les\\n                            seules r\\u00e9alisations mat\\u00e9rielles ne nous combleront pas longtemps. En\\n                            v\\u00e9rit\\u00e9, la poursuite aveugle de l\\u2019argent, de la renomm\\u00e9e et du plaisir\\n                            nous rendra seulement mis\\u00e9rables. \\u00c9picure recommandait par exemple de\\n                            manger et de boire avec mod\\u00e9ration, et de r\\u00e9fr\\u00e9ner ses app\\u00e9tits sexuels.\\n                            \\u00c0 la longue, une amiti\\u00e9 profonde nous \\u00e9panouira davantage qu\\u2019une orgie\\n                            fr\\u00e9n\\u00e9tique. \\u00c9picure \\u00e9baucha toute une \\u00e9thique du \\u00ab\\u00a0fais et ne fais\\n                            pas\\u00a0\\u00bb pour guider les hommes sur la voie sem\\u00e9e d\\u2019emb\\u00fbches du\\n                            bonheur.", "cluster": -1}, {"text": "\\u00c9picure \\u00e9tait apparemment sur la voie de quelque chose. \\u00catre heureux\\n                            n\\u2019est pas si facile. Malgr\\u00e9 les r\\u00e9alisations sans pr\\u00e9c\\u00e9dent des toutes\\n                            derni\\u00e8res d\\u00e9cennies, il est loin d\\u2019\\u00eatre \\u00e9vident que nos contemporains\\n                            sont sensiblement plus satisfaits que leurs anc\\u00eatres du temps jadis. Il\\n                            est en fait un signe de mauvais augure\\u00a0: malgr\\u00e9 une prosp\\u00e9rit\\u00e9, un\\n                            confort et une s\\u00e9curit\\u00e9 accrus, le taux de suicide dans le monde\\n                            d\\u00e9velopp\\u00e9 est bien plus \\u00e9lev\\u00e9 que dans les soci\\u00e9t\\u00e9s traditionnelles.", "cluster": -1}, {"text": "Au P\\u00e9rou, en Ha\\u00efti, aux Philippines et au Ghana, pays en voie de\\n                            d\\u00e9veloppement qui souffrent de pauvret\\u00e9 et d\\u2019instabilit\\u00e9 politique,\\n                            moins de cinq personnes sur cent\\u00a0mille se suicident chaque ann\\u00e9e. Dans\\n                            des pays riches et calmes comme la Suisse, la France, le Japon etla Nouvelle-Z\\u00e9lande, plus de dix personnes sur\\n                            cent\\u00a0mille se donnent la mort chaque ann\\u00e9e. En 1985, la Cor\\u00e9e du Sud\\n                            \\u00e9tait pauvre, r\\u00e9gie par la tradition et soumise \\u00e0 une dictature.\\n                            Aujourd\\u2019hui, la Cor\\u00e9e du Sud est une puissance \\u00e9conomique de premier\\n                            plan, ses citoyens comptent parmi les mieux \\u00e9duqu\\u00e9s du monde et\\n                            jouissent d\\u2019un r\\u00e9gime d\\u00e9mocratique stable et relativement lib\\u00e9ral. Mais,\\n                            alors qu\\u2019en 1985 autour de neuf Sud-Cor\\u00e9ens sur cent\\u00a0mille se donnaient\\n                            la mort, le taux annuel de suicide est de trente-six pour cent\\u00a0mille\\n                                aujourd\\u2019hui(32).", "cluster": -1}, {"text": "Il y a bien entendu des tendances contraires et bien plus encourageantes.\\n                            La tr\\u00e8s forte diminution de la mortalit\\u00e9 infantile s\\u2019est tr\\u00e8s\\n                            certainement sold\\u00e9e par une augmentation du bonheur humain et a en\\n                            partie compens\\u00e9 le stress de la vie moderne. Reste que, m\\u00eame si nous\\n                            sommes un peu plus heureux que nos anc\\u00eatres, l\\u2019accroissement de notre\\n                            bien-\\u00eatre est bien plus modeste que nous aurions pu l\\u2019esp\\u00e9rer. \\u00c0 l\\u2019\\u00e2ge\\n                            de pierre, l\\u2019homme moyen disposait d\\u2019environ 4\\u00a0000\\u00a0calories d\\u2019\\u00e9nergie\\n                            par jour\\u00a0: ce qui comprenait non seulement la nourriture, mais\\n                            aussi l\\u2019\\u00e9nergie investie dans la confection d\\u2019outils, de v\\u00eatements,\\n                            d\\u2019art et de feux de camp. Aujourd\\u2019hui, l\\u2019Am\\u00e9ricain moyen d\\u00e9pense\\n                            228\\u00a0000\\u00a0calories d\\u2019\\u00e9nergie par jour pour remplir son estomac, mais aussi\\n                            sa voiture, son ordinateur, son r\\u00e9frig\\u00e9rateur et sa t\\u00e9l\\u00e9vision(33). L\\u2019Am\\u00e9ricain moyen d\\u00e9pense soixante fois plus\\n                            d\\u2019\\u00e9nergie que le chasseur-cueilleur moyen de l\\u2019\\u00e2ge de pierre. Est-il\\n                            pour autant soixante fois plus heureux\\u00a0? Un tel optimisme peut\\n                            bien nous laisser sceptiques.", "cluster": -1}, {"text": "Et m\\u00eame si nous avons surmont\\u00e9 nombre des fl\\u00e9aux d\\u2019antan, il est sans\\n                            doute bien plus difficile d\\u2019atteindre un bonheur total que d\\u2019abolir\\n                            carr\\u00e9ment la souffrance. Un morceau de pain suffisait \\u00e0 la joie d\\u2019un\\n                            paysan affam\\u00e9 du Moyen \\u00c2ge. Comment procurer de la joie \\u00e0 un ing\\u00e9nieur\\n                            fatigu\\u00e9, surpay\\u00e9 et en surpoids\\u00a0? La seconde moiti\\u00e9 duXXesi\\u00e8cle a \\u00e9t\\u00e9 un \\u00e2ge d\\u2019or pour les\\n                            \\u00c9tats-Unis. Leur victoire dans la Seconde Guerre mondiale, suivie d\\u2019une\\n                            victoire encore plus d\\u00e9cisive dans la guerre froide, en a fait la\\n                            premi\\u00e8re superpuissance du monde. Entre 1950 et 2000, le PIB am\\u00e9ricain\\n                            est pass\\u00e9 de 2 \\u00e0 12\\u00a0billions de dollars. Le revenu r\\u00e9el par t\\u00eate a\\n                            doubl\\u00e9. La toute nouvelle pilule contraceptive a rendu la sexualit\\u00e9 plus\\n                            lib\\u00e9r\\u00e9e que jamais. Les femmes, les gays, les Afro-Am\\u00e9ricains et les\\n                            autres minorit\\u00e9s ont fini par obtenir une plus grosse part du g\\u00e2teau\\n                            am\\u00e9ricain. Une foule de produits bon march\\u00e9\\u2013\\u00a0voitures,\\n                            r\\u00e9frig\\u00e9rateurs, air conditionn\\u00e9, aspirateurs, lave-vaisselle, machines \\u00e0\\n                            laver, t\\u00e9l\\u00e9phones, t\\u00e9l\\u00e9viseurs et ordinateurs\\u00a0\\u2013 ont chang\\u00e9 la vie\\n                            quotidienne, au point de la rendre presque m\\u00e9connaissable. Des \\u00e9tudes\\n                            ont pourtant montr\\u00e9 que les niveaux de bien-\\u00eatre subjectifs am\\u00e9ricains\\n                            dans les ann\\u00e9es\\u00a01990 sont rest\\u00e9s \\u00e0 peu pr\\u00e8s les m\\u00eames que dans les\\n                                ann\\u00e9es\\u00a01950(34).", "cluster": -1}, {"text": "Au Japon, le revenu r\\u00e9el moyen a \\u00e9t\\u00e9 multipli\\u00e9 par cinq entre 1958 et\\n                            1987, dans l\\u2019un des essors \\u00e9conomiques les plus rapides de l\\u2019histoire.\\n                            Cette avalanche de richesse, de concert avec une myriade de changements\\n                            positifs, et n\\u00e9gatifs, de modes de vie et de relations sociales, a eu\\n                            \\u00e9tonnamment peu d\\u2019impact sur les niveaux de bien-\\u00eatre subjectifs. Dans\\n                            les ann\\u00e9es\\u00a01990, les Japonais \\u00e9taient aussi satisfaits \\u2013\\u00a0ou\\n                            insatisfaits\\u00a0\\u2013 que dans les ann\\u00e9es\\u00a01950(35).", "cluster": -1}, {"text": "Il semble que notre bonheur se heurte \\u00e0 un myst\\u00e9rieux plafond de verre\\n                            qui ne lui permet pas de cro\\u00eetre, malgr\\u00e9 nos r\\u00e9alisations sans\\n                            pr\\u00e9c\\u00e9dent. M\\u00eame si nous fournissions de la nourriture gratuite \\u00e0 tout le\\n                            monde, soignions toutes les maladies et instaurions la paix dans le\\n                            monde, cela ne briserait pas n\\u00e9cessairement le plafond de verre. Trouver\\n                            le vrai bonheur ne sera pas beaucoup plus facile que surmonter la\\n                            vieillesse et la mort.", "cluster": 148}, {"text": "Le plafond de verre du bonheur s\\u2019appuie sur deux robustes piliers\\u00a0:\\n                            l\\u2019un psychologique, l\\u2019autre biologique. Au niveau psychologique, le\\n                            bonheur d\\u00e9pend des attentes plut\\u00f4t que de conditions objectives. Ce\\n                            n\\u2019est pas de mener une vie paisible et prosp\\u00e8re qui nous procure\\n                            satisfaction. Nous sommes plut\\u00f4t satisfaits quand la r\\u00e9alit\\u00e9 r\\u00e9pond \\u00e0\\n                            nos attentes. La mauvaise nouvelle, c\\u2019est que les attentes s\\u2019envolent \\u00e0\\n                            mesure que les conditions s\\u2019am\\u00e9liorent. L\\u2019am\\u00e9lioration spectaculaire des\\n                            conditions de vie que l\\u2019humanit\\u00e9 a connue dans les derni\\u00e8res d\\u00e9cennies\\n                            se solde par de plus fortes attentes, plut\\u00f4t que par un plus grand\\n                            contentement. Si nous n\\u2019y faisons rien, nos r\\u00e9alisations futures\\n                            pourraient bien nous laisser tout aussi insatisfaits.", "cluster": 148}, {"text": "Au niveau biologique, c\\u2019est notre biochimie, plut\\u00f4t que notre situation\\n                            politique, sociale et \\u00e9conomique, qui d\\u00e9termine \\u00e0 la fois nos attentes\\n                            et notre bonheur. Selon \\u00c9picure, nous sommes heureux quand nous\\n                            \\u00e9prouvons des sensations plaisantes et \\u00e9chappons aux sensations\\n                            d\\u00e9sagr\\u00e9ables. Jeremy Bentham soutenait pareillement que la nature adonn\\u00e9 \\u00e0 deux ma\\u00eetres \\u2013\\u00a0le plaisir et la peine\\u00a0\\u2013 un\\n                            empire sur l\\u2019homme et qu\\u2019eux seuls d\\u00e9terminent tout ce que nous faisons,\\n                            disons et pensons. John Stuart Mill, le successeur de Bentham, expliqua\\n                            que le bonheur n\\u2019est rien d\\u2019autre que le plaisir et l\\u2019absence de\\n                            douleur, et que, par-del\\u00e0 le plaisir et la peine, il n\\u2019est ni bien ni\\n                            mal. Qui essaie de d\\u00e9duire le bien et le mal d\\u2019autre chose \\u2013\\u00a0comme la\\n                            parole de Dieu ou l\\u2019int\\u00e9r\\u00eat national\\u00a0\\u2013 vous dupe et se dupe peut-\\u00eatre\\n                            lui aussi par la m\\u00eame occasion(36).", "cluster": 222}, {"text": "Au temps d\\u2019\\u00c9picure, un tel discours \\u00e9tait blasph\\u00e9matoire. Au temps de\\n                            Bentham et de Mill, il \\u00e9tait hautement subversif. \\u00c0 l\\u2019aube duXXIesi\\u00e8cle, il est devenu l\\u2019orthodoxie\\n                            scientifique. Suivant les sciences de la vie, bonheur et souffrance ne\\n                            sont que des \\u00e9quilibres diff\\u00e9rents de sensations corporelles. Nous ne\\n                            r\\u00e9agissons jamais aux \\u00e9v\\u00e9nements du monde ext\\u00e9rieur, seulement aux\\n                            sensations physiques. On ne souffre pas d\\u2019avoir perdu son emploi ou\\n                            divorc\\u00e9, ou parce que le gouvernement a d\\u00e9clar\\u00e9 la guerre. La seule\\n                            chose qui rende les gens malheureux, ce sont les sensations d\\u00e9sagr\\u00e9ables\\n                            \\u00e9prouv\\u00e9es par leur corps. Perdre son travail peut certainement\\n                            d\\u00e9clencher une d\\u00e9pression, mais la d\\u00e9pression est elle-m\\u00eame une sorte de\\n                            sensation physique d\\u00e9sagr\\u00e9able. Un millier de choses peuvent nous\\n                            f\\u00e2cher, mais la col\\u00e8re n\\u2019est jamais une abstraction. Elle est toujours\\n                            \\u00e9prouv\\u00e9e comme une sensation de chaleur et de tension dans le corps,\\n                            c\\u2019est pourquoi elle peut vous mettre en rage, d\\u2019o\\u00f9 l\\u2019expression\\n                            \\u00ab\\u00a0br\\u00fbler de col\\u00e8re\\u00a0\\u00bb.", "cluster": -1}, {"text": "Inversement, la science nous dit que ce n\\u2019est pas obtenir une promotion,\\n                            gagner \\u00e0 la loterie ou m\\u00eame trouver le grand amour qui rend heureux. Il\\n                            est une seule chose, et une chose seulement, qui rende les gens\\n                            heureux\\u00a0: les sensations agr\\u00e9ables. Imaginez-vous dans la peau de\\n                            Mario G\\u00f6tze, le milieu de terrain de l\\u2019\\u00e9quipe allemande de football,\\n                            pendant la finale de la Coupe du monde de 2014 contre\\n                            l\\u2019Argentine\\u00a0; cent treize minutes se sont d\\u00e9j\\u00e0 \\u00e9coul\\u00e9es, et le\\n                            score est toujours de z\\u00e9ro \\u00e0 z\\u00e9ro. Il ne reste plus que sept minutes\\n                            avant la redoutable \\u00e9preuve des tirs au but. Soixante-quinze mille fans\\n                            excit\\u00e9s se pressent dans le stade Maracan\\u00e3 de Rio, des millions et des\\n                            millions de t\\u00e9l\\u00e9spectateurs \\u00e0 travers le monde suivent le match en\\n                            retenant leur souffle. Vous \\u00eates \\u00e0 quelques m\\u00e8tres des buts argentins,\\n                            quand Andr\\u00e9 Sch\\u00fcrrle fait une magnifique passe dans votre direction.\\n                            Vous contr\\u00f4lez le ballon de la poitrine, il retombe vers votre jambe,\\n                            vous le frappez de vol\\u00e9e et vousle voyez passer devant\\n                            le gardien argentin puis s\\u2019enfoncer dans les filets.Goooooooal\\u00a0!Le stade est en \\u00e9ruption comme un volcan.\\n                            Des dizaines de milliers de gens hurlent comme des fous, vos co\\u00e9quipiers\\n                            courent pour vous enlacer et vous embrasser\\u00a0; \\u00e0 Berlin et \\u00e0\\n                            Munich, des millions de gens versent des larmes devant leur petit \\u00e9cran.\\n                            Vous \\u00eates en extase, mais ce n\\u2019est pas \\u00e0 cause du ballon dans les filets\\n                            argentins ni \\u00e0 cause des r\\u00e9jouissances qui se poursuivent dans lesBiergartensbavarois. Vous r\\u00e9agissez en fait \\u00e0 la temp\\u00eate de\\n                            sensations qui se d\\u00e9cha\\u00eene en vous. Vous avez des frissons dans le dos,\\n                            des ondes \\u00e9lectriques inondent votre corps, vous avez l\\u2019impression de\\n                            vous dissoudre en millions de boules d\\u2019\\u00e9nergie qui explosent.", "cluster": 134}, {"text": "Il n\\u2019est pas n\\u00e9cessaire de marquer le but de la victoire dans la Coupe du\\n                            monde pour \\u00e9prouver de telles sensations. Si vous recevez une promotion\\n                            inattendue au travail et que vous vous mettiez \\u00e0 sauter de joie, vous\\n                            r\\u00e9agissez au m\\u00eame genre de sensations. Les parties les plus profondes de\\n                            votre esprit ne savent rien du football ou des autres m\\u00e9tiers. Elles ne\\n                            connaissent que les sensations. Si vous \\u00eates promu, mais que, pour une\\n                            raison ou pour une autre, vous n\\u2019aviez pas de sensations plaisantes,\\n                            vous ne serez pas content. Si vous veniez d\\u2019\\u00eatre vir\\u00e9 (ou de perdre un\\n                            match de foot d\\u00e9cisif), mais que vous ressentiez des sensations tr\\u00e8s\\n                            agr\\u00e9ables (vous avez peut-\\u00eatre ing\\u00e9r\\u00e9 telle ou telle substance), vous\\n                            pourriez bien vous sentir au septi\\u00e8me ciel.", "cluster": 134}, {"text": "La mauvaise nouvelle, c\\u2019est que les sensations agr\\u00e9ables retombent vite.\\n                            T\\u00f4t ou tard, elles laissent place \\u00e0 d\\u2019autres, d\\u00e9sagr\\u00e9ables. M\\u00eame marquer\\n                            le but de la victoire en finale de la Coupe du monde n\\u2019est pas un gage\\n                            de f\\u00e9licit\\u00e9 durable. En fait, \\u00e7a pourrait m\\u00eame bien \\u00eatre la fin des\\n                            haricots. De m\\u00eame si, l\\u2019an dernier, j\\u2019ai b\\u00e9n\\u00e9fici\\u00e9 d\\u2019une promotion\\n                            inattendue, je pourrais bien occuper ce nouveau poste alors que les\\n                            sensations tr\\u00e8s agr\\u00e9ables que j\\u2019ai \\u00e9prouv\\u00e9es en apprenant la nouvelle se\\n                            sont effac\\u00e9es en l\\u2019espace de quelques heures. Si je veux retrouver ces\\n                            merveilleuses sensations, je dois d\\u00e9crocher une nouvelle promotion. Puis\\n                            encore une autre. Si je ne l\\u2019obtiens pas, je pourrais bien finir\\n                            beaucoup plus amer et contrari\\u00e9 que si j\\u2019\\u00e9tais rest\\u00e9 un simple pion.", "cluster": 134}, {"text": "Tout cela est la faute de l\\u2019\\u00e9volution. Depuis des g\\u00e9n\\u00e9rations, notre\\n                            syst\\u00e8me biochimique s\\u2019est adapt\\u00e9 de mani\\u00e8re \\u00e0 accro\\u00eetre nos chances de\\n                            survie et de reproduction, mais pas notre bonheur. Le syst\\u00e8mebiochimique r\\u00e9compense les actions propices \\u00e0 la survie\\n                            et \\u00e0 la reproduction par des sensations agr\\u00e9ables. Mais ce n\\u2019est qu\\u2019une\\n                            astuce \\u00e9ph\\u00e9m\\u00e8re pour nous motiver. Nous nous battons pour obtenir de la\\n                            nourriture et des partenaires afin d\\u2019\\u00e9viter les sensations d\\u00e9plaisantes\\n                            que procure la faim et afin de jouir de go\\u00fbts agr\\u00e9ables et de d\\u00e9licieux\\n                            orgasmes. Or ces go\\u00fbts et ces orgasmes ne durent pas tr\\u00e8s longtemps, et\\n                            si nous voulons les ressentir \\u00e0 nouveau, nous devons rechercher\\n                            davantage de vivres et de partenaires.", "cluster": 221}, {"text": "Que se serait-il produit si une mutation rare avait cr\\u00e9\\u00e9 un \\u00e9cureuil qui,\\n                            apr\\u00e8s avoir mang\\u00e9 une noisette, jouissait d\\u2019une sensation de f\\u00e9licit\\u00e9\\n                            \\u00e9ternelle\\u00a0? Techniquement, cela pourrait se produire en rec\\u00e2blant\\n                            le cerveau de l\\u2019\\u00e9cureuil. Qui sait\\u00a0? Peut-\\u00eatre est-ce arriv\\u00e9 voici\\n                            des millions d\\u2019ann\\u00e9es \\u00e0 un \\u00e9cureuil heureux\\u00a0? Mais, en ce cas, cet\\n                            \\u00e9cureuil a joui d\\u2019une vie aussi heureuse que br\\u00e8ve, et cela a sign\\u00e9 la\\n                            fin de cette mutation rare. Car l\\u2019\\u00e9cureuil b\\u00e9at ne se serait pas donn\\u00e9\\n                            la peine de chercher d\\u2019autres noisettes, encore moins d\\u2019autres\\n                            partenaires. Les \\u00e9cureuils rivaux, qui avaient de nouveau faim cinq\\n                            minutes apr\\u00e8s avoir grignot\\u00e9 une noisette, avaient de bien meilleures\\n                            chances de survivre et de transmettre leurs g\\u00e8nes \\u00e0 la g\\u00e9n\\u00e9ration\\n                            suivante. Pour la m\\u00eame raison exactement, les noisettes que nous, les\\n                            hommes, cherchons \\u00e0 ramasser \\u2013\\u00a0emplois lucratifs, grandes maisons, beaux\\n                            partenaires\\u00a0\\u2013 nous satisfont rarement longtemps.", "cluster": -1}, {"text": "D\\u2019aucuns diront sans doute que ce n\\u2019est pas si mal, que ce n\\u2019est pas le\\n                            but qui rend heureux, mais le voyage. L\\u2019ascension de l\\u2019Everest procure\\n                            plus de satisfaction que de se tenir \\u00e0 son sommet\\u00a0; la s\\u00e9duction\\n                            et les pr\\u00e9liminaires sont plus excitants que l\\u2019orgasme\\u00a0; et mener\\n                            des exp\\u00e9riences r\\u00e9volutionnaires en laboratoire est plus int\\u00e9ressant\\n                            qu\\u2019\\u00eatre couvert de louanges et de r\\u00e9compenses. Cela ne change rien \\u00e0 la\\n                            situation. Et reste le signe que l\\u2019\\u00e9volution nous contr\\u00f4le par un large\\n                            \\u00e9ventail de plaisirs. Tant\\u00f4t elle nous s\\u00e9duit par de douces sensations\\n                            de f\\u00e9licit\\u00e9 et de tranquillit\\u00e9, tant\\u00f4t elle nous aiguillonne par des\\n                            sensations grisantes d\\u2019all\\u00e9gresse et d\\u2019excitation.", "cluster": 221}, {"text": "Quand une cr\\u00e9ature recherche quelque chose qui accroisse ses chances de\\n                            survie et de reproduction \\u2013\\u00a0par exemple, de la nourriture, des\\n                            partenaires ou un statut social\\u00a0\\u2013, son cerveau produit des sensations de\\n                            vigilance et d\\u2019excitation qui la poussent \\u00e0 des efforts toujours plusgrands tant ils sont agr\\u00e9ables. Dans une exp\\u00e9rience\\n                            c\\u00e9l\\u00e8bre, des chercheurs ont branch\\u00e9 des \\u00e9lectrodes aux cerveaux de\\n                            plusieurs rats, permettant \\u00e0 ceux-ci de d\\u00e9clencher des sensations\\n                            d\\u2019excitation en appuyant simplement sur une p\\u00e9dale. Quand on donnait aux\\n                            rats le choix entre des aliments savoureux et appuyer sur la p\\u00e9dale, ils\\n                            pr\\u00e9f\\u00e9raient la seconde solution (un peu comme les enfants qui pr\\u00e9f\\u00e8rent\\n                            les jeux vid\\u00e9o plut\\u00f4t que de se mettre \\u00e0 table). Les rats ne cessaient\\n                            de presser sur la p\\u00e9dale, jusqu\\u2019\\u00e0 s\\u2019\\u00e9crouler de faim et\\n                                d\\u2019\\u00e9puisement(37). Les hommes aussi pr\\u00e9f\\u00e8rent sans\\n                            doute l\\u2019excitation de la course au repos sur les lauriers de la\\n                            r\\u00e9ussite. Pourtant, ce qui rend la course si attrayante, ce sont\\n                            pr\\u00e9cis\\u00e9ment les sensations grisantes qui l\\u2019accompagnent. Personne ne\\n                            voudrait escalader des montagnes, jouer \\u00e0 des jeux vid\\u00e9o ou se rendre \\u00e0\\n                            desblind dates(pour rencontrer des inconnus) si ces activit\\u00e9s\\n                            s\\u2019accompagnaient exclusivement de sensations d\\u00e9plaisantes de stress, de\\n                            d\\u00e9sespoir ou d\\u2019ennui(38).", "cluster": 87}, {"text": "Les sensations excitantes de la course sont h\\u00e9las aussi transitoires que\\n                            les sensations de f\\u00e9licit\\u00e9 provoqu\\u00e9es par la victoire. Le don Juan qui\\n                            jouit du frisson d\\u2019une aventure sans lendemain, l\\u2019homme d\\u2019affaires qui\\n                            jouit de se ronger les ongles en voyant le Dow Jones monter et baisser,\\n                            et le joueur qui jouit de tuer des monstres sur son \\u00e9cran d\\u2019ordinateur\\n                            ne trouveront aucune satisfaction \\u00e0 se souvenir de leurs aventures de la\\n                            veille. Comme les rats qui appuient sans cesse sur la p\\u00e9dale, les don\\n                            Juan, les magnats et les joueurs ont besoin chaque jour d\\u2019un nouvel\\n                            aiguillon. Pire encore, ici aussi les attentes s\\u2019adaptent aux\\n                            conditions, et les d\\u00e9fis d\\u2019hier ne deviennent que trop rapidement\\n                            l\\u2019ennui d\\u2019aujourd\\u2019hui. Peut-\\u00eatre la cl\\u00e9 du bonheur n\\u2019est-elle ni la\\n                            course ni la m\\u00e9daille d\\u2019or, mais plut\\u00f4t la combinaison d\\u2019un bon dosage\\n                            d\\u2019excitation et de tranquillit\\u00e9\\u00a0; la plupart d\\u2019entre nous avons\\n                            tendance \\u00e0 bondir du stress \\u00e0 l\\u2019ennui et inversement pour demeurer aussi\\n                            insatisfaits de l\\u2019un que de l\\u2019autre.", "cluster": 87}, {"text": "Si la science a raison et que notre syst\\u00e8me biochimique d\\u00e9termine notre\\n                            bonheur, l\\u2019unique mani\\u00e8re d\\u2019assurer une satisfaction durable est de\\n                            truquer ce syst\\u00e8me. Oublions la croissance \\u00e9conomique, les r\\u00e9formes\\n                            sociales et les r\\u00e9volutions politiques\\u00a0: pour \\u00e9lever le niveau\\n                            g\\u00e9n\\u00e9ral de bonheur, il nous faut manipuler la biochimie humaine. Et\\n                            c\\u2019est exactement ce que nous avons commenc\\u00e9 \\u00e0 faire depuis quelques\\n                            d\\u00e9cennies.Voici un demi-si\\u00e8cle, les psychotropes\\n                            \\u00e9taient stigmatis\\u00e9s. Ce n\\u2019est plus le cas de nos jours. Pour le meilleur\\n                            ou pour le pire, un pourcentage croissant de la population en prend\\n                            r\\u00e9guli\\u00e8rement pour soigner des maladies mentales handicapantes, mais\\n                            aussi affronter des d\\u00e9pressions plus prosa\\u00efques et des coups de\\n                            cafard.", "cluster": 225}, {"text": "Par exemple, un nombre croissant d\\u2019\\u00e9coliers prend des stimulants comme la\\n                            Ritaline. En 2011, 3,\\u00a05\\u00a0millions d\\u2019enfants am\\u00e9ricains prenaient des\\n                            m\\u00e9dicaments pour TDAH (troubles du d\\u00e9ficit de l\\u2019attention et\\n                            hyperactivit\\u00e9). Au Royaume-Uni, leur nombre est pass\\u00e9 de 92\\u00a0000 en 1997\\n                            \\u00e0 786\\u00a0000 en 2012(39). L\\u2019objectif initial \\u00e9tait de\\n                            traiter les troubles de l\\u2019attention, mais aujourd\\u2019hui des enfants en\\n                            parfaite sant\\u00e9 prennent ce type de m\\u00e9dicaments pour am\\u00e9liorer leurs\\n                            r\\u00e9sultats scolaires et \\u00eatre \\u00e0 la hauteur des attentes croissantes des\\n                            enseignants et des parents(40). Beaucoup critiquent\\n                            cette \\u00e9volution et soutiennent que le probl\\u00e8me vient du syst\\u00e8me \\u00e9ducatif\\n                            plut\\u00f4t que des enfants. Si les \\u00e9l\\u00e8ves souffrent de troubles de\\n                            l\\u2019attention, de stress et de mauvaises notes, peut-\\u00eatre devrions-nous\\n                            bl\\u00e2mer des m\\u00e9thodes p\\u00e9dagogiques surann\\u00e9es, des classes surpeupl\\u00e9es et\\n                            un rythme de vie anormalement rapide. Peut-\\u00eatre devrions-nous modifier\\n                            les \\u00e9coles plut\\u00f4t que les enfants\\u00a0? Il est int\\u00e9ressant de voir\\n                            comment les arguments ont \\u00e9volu\\u00e9. Les querelles autour des m\\u00e9thodes\\n                            \\u00e9ducatives remontent \\u00e0 des milliers d\\u2019ann\\u00e9es. En Chine ancienne comme\\n                            dans la Grande-Bretagne victorienne, chacun privil\\u00e9giait sa m\\u00e9thode et\\n                            s\\u2019opposait avec v\\u00e9h\\u00e9mence \\u00e0 toutes les autres solutions. Jusqu\\u2019ici,\\n                            pourtant, tout le monde s\\u2019entendait encore sur un point\\u00a0: pour\\n                            am\\u00e9liorer l\\u2019\\u00e9ducation, il nous faut changer les \\u00e9coles. Aujourd\\u2019hui,\\n                            pour la premi\\u00e8re fois dans l\\u2019histoire, certaines personnes au moins\\n                            pensent qu\\u2019il serait plus efficace de modifier la biochimie des\\n                                \\u00e9l\\u00e8ves(41).", "cluster": 324}, {"text": "Les arm\\u00e9es suivent la m\\u00eame voie\\u00a0: 12\\u00a0% des soldats am\\u00e9ricains en\\n                            Irak et 17\\u00a0% des soldats am\\u00e9ricains en Afghanistan prenaient des\\n                            somnif\\u00e8res ou des antid\\u00e9presseurs qui les aidaient \\u00e0 affronter la\\n                            pression et la d\\u00e9tresse de la guerre. La peur, la d\\u00e9pression et les\\n                            traumatismes ne sont pas caus\\u00e9s par les obus, les colis ou les voitures\\n                            pi\\u00e9g\\u00e9s, mais par les hormones, les neurotransmetteurs et les r\\u00e9seaux\\n                            neuronaux. Deux soldats peuvent se trouver au coude \\u00e0 coude dans une\\n                            embuscade\\u00a0; l\\u2019un se figera de terreur, perdra l\\u2019esprit et\\n                            souffrira de cauchemars desann\\u00e9es apr\\u00e8s les\\n                            faits\\u00a0; l\\u2019autre chargera courageusement et d\\u00e9crochera une\\n                            m\\u00e9daille. La diff\\u00e9rence est dans la biochimie des soldats, et si nous\\n                            trouvons le moyen de la ma\\u00eetriser, elle produira d\\u2019un seul coup des\\n                            soldats plus heureux et des arm\\u00e9es plus efficaces(42).", "cluster": -1}, {"text": "La poursuite biochimique du bonheur est aussi la cause num\\u00e9ro un du crime\\n                            dans le monde. En 2009, la moiti\\u00e9 des d\\u00e9tenus des prisons f\\u00e9d\\u00e9rales\\n                            am\\u00e9ricaines \\u00e9taient intern\\u00e9s pour des affaires de drogue\\u00a0; 38\\u00a0%\\n                            des prisonniers italiens \\u00e9taient condamn\\u00e9s pour des crimes en lien avec\\n                            des stup\\u00e9fiants\\u00a0; 55\\u00a0% des d\\u00e9tenus du Royaume-Uni disaient avoir\\n                            commis des crimes en rapport avec la consommation ou le trafic de\\n                            drogue. Selon un rapport de 2001, 62\\u00a0% des condamn\\u00e9s australiens \\u00e9taient\\n                            sous l\\u2019empire de la drogue quand ils ont commis le crime qui leur a valu\\n                            d\\u2019\\u00eatre incarc\\u00e9r\\u00e9s(43).", "cluster": -1}, {"text": "Les gens boivent de l\\u2019alcool pour oublier, fument pour se d\\u00e9tendre,\\n                            prennent de la coca\\u00efne ou de la m\\u00e9thamph\\u00e9tamine pour \\u00eatre alertes et\\n                            s\\u00fbrs d\\u2019eux, tandis que l\\u2019ecstasy procure des sensations\\n                            \\u00ab\\u00a0extatiques\\u00a0\\u00bb et que le LSD vous envoie retrouver\\n                            \\u00ab\\u00a0Lucy in the Sky with Diamonds\\u00a0\\u00bb . Ce que les uns esp\\u00e8rent\\n                            trouver en \\u00e9tudiant, en travaillant ou en \\u00e9levant une famille, d\\u2019autres\\n                            essaient de l\\u2019obtenir bien plus facilement par un bon dosage de\\n                            mol\\u00e9cules. C\\u2019est une menace existentielle pour l\\u2019ordre \\u00e9conomique et\\n                            social, lequel pousse les pays \\u00e0 livrer une guerre obstin\\u00e9e, sanglante\\n                            et d\\u00e9sesp\\u00e9r\\u00e9e au crime biochimique.", "cluster": -1}, {"text": "L\\u2019\\u00c9tat esp\\u00e8re r\\u00e9guler la poursuite biochimique du bonheur en s\\u00e9parant les\\n                            \\u00ab\\u00a0mauvaises\\u00a0\\u00bb manipulations des \\u00ab\\u00a0bonnes\\u00a0\\u00bb. Le\\n                            principe est clair\\u00a0: les manipulations biochimiques qui renforcent\\n                            la stabilit\\u00e9 politique, l\\u2019ordre social et la croissance \\u00e9conomique sont\\n                            autoris\\u00e9es, voire encourag\\u00e9es (par exemple, celles qui calment les\\n                            enfants hyperactifs \\u00e0 l\\u2019\\u00e9cole ou poussent les soldats anxieux \\u00e0 engager\\n                            le combat). Les manipulations qui menacent la stabilit\\u00e9 et la croissance\\n                            sont interdites. Chaque ann\\u00e9e, cependant, les laboratoires de recherche\\n                            des universit\\u00e9s, des compagnies pharmaceutiques et des organisations\\n                            criminelles inventent de nouvelles substances\\u00a0; de m\\u00eame, les\\n                            besoins de l\\u2019\\u00c9tat et du march\\u00e9 ne cessent de changer. La poursuite\\n                            biochimique du bonheur s\\u2019acc\\u00e9l\\u00e9rant, elle refa\\u00e7onnera la politique, la\\n                            soci\\u00e9t\\u00e9 et l\\u2019\\u00e9conomie, et il deviendra toujours plus difficile de la\\n                            dominer.", "cluster": 225}, {"text": "Les drogues ne sont qu\\u2019un d\\u00e9but. Dans les labos de\\n                            recherche, les experts travaillent d\\u00e9j\\u00e0 sur des mani\\u00e8res plus\\n                            sophistiqu\\u00e9es de manipuler la biochimie humaine, en envoyant des stimuli\\n                            directs \\u00e0 des points pr\\u00e9cis du cerveau ou, par le g\\u00e9nie g\\u00e9n\\u00e9tique, en\\n                            manipulant la trame m\\u00eame du corps. Qu\\u2019importe la m\\u00e9thode\\u00a0: arriver\\n                            au bonheur par une manipulation g\\u00e9n\\u00e9tique ne sera pas facile, car cela\\n                            exige de changer les structures fondamentales du vivant. Mais il est\\n                            vrai qu\\u2019il n\\u2019a pas \\u00e9t\\u00e9 facile de triompher de la famine, des \\u00e9pid\\u00e9mies\\n                            et de la guerre.", "cluster": 225}, {"text": "Que l\\u2019humanit\\u00e9 doive investir tant d\\u2019efforts dans la poursuite\\n                            biochimique du bonheur est loin d\\u2019\\u00eatre une certitude. D\\u2019aucuns diraient\\n                            que le bonheur, au fond, n\\u2019a pas tant d\\u2019importance, qu\\u2019on a tort de\\n                            faire de la satisfaction individuelle le but supr\\u00eame de la soci\\u00e9t\\u00e9.\\n                            D\\u2019autres admettent que le bonheur est effectivement le bien supr\\u00eame,\\n                            tout en contestant la d\\u00e9finition biologique du bonheur comme exp\\u00e9rience\\n                            de sensations plaisantes.", "cluster": 222}, {"text": "Il y a deux mille trois cents ans, \\u00c9picure avertit ses disciples\\u00a0:\\n                            la poursuite immod\\u00e9r\\u00e9e du plaisir pourrait bien les rendre non pas\\n                            heureux, mais mis\\u00e9rables. Deux si\\u00e8cles auparavant, Bouddha avait soutenu\\n                            une id\\u00e9e encore plus radicale, affirmant que la poursuite des sensations\\n                            agr\\u00e9ables \\u00e9tait en fait la racine m\\u00eame de la souffrance. Ces sensations\\n                            ne sont que des vibrations \\u00e9ph\\u00e9m\\u00e8res et d\\u00e9nu\\u00e9es de sens. M\\u00eame quand nous\\n                            en faisons l\\u2019exp\\u00e9rience, notre r\\u00e9action n\\u2019est pas le contentement, mais\\n                            le d\\u00e9sir de toujours plus. D\\u00e8s lors, j\\u2019ai beau \\u00e9prouver toujours plus de\\n                            sensations de f\\u00e9licit\\u00e9 ou d\\u2019excitation, jamais elles ne me\\n                            satisferont.", "cluster": 150}, {"text": "Si j\\u2019identifie le bonheur \\u00e0 des sensations agr\\u00e9ables et fugitives, et\\n                            d\\u00e9sire ardemment en \\u00e9prouver toujours plus, je n\\u2019ai d\\u2019autre choix que de\\n                            les poursuivre constamment. Lorsque je finis par les obtenir, elles\\n                            disparaissent rapidement\\u00a0; le simple souvenir des plaisirs pass\\u00e9s\\n                            ne me donnera pas satisfaction et je devrai tout recommencer. M\\u00eame si je\\n                            poursuis cette qu\\u00eate des d\\u00e9cennies durant, elle ne me vaudra jamais\\n                            aucune r\\u00e9alisation durable\\u00a0; bien au contraire, plus je d\\u00e9sirerai\\n                            ces sensations plaisantes, plus je deviendrai angoiss\\u00e9 et insatisfait.\\n                            Pouratteindre le vrai bonheur, les humains doivent\\n                            ralentir leur qu\\u00eate de sensations agr\\u00e9ables, non pas l\\u2019acc\\u00e9l\\u00e9rer.", "cluster": 150}, {"text": "Cette vision bouddhiste du bonheur a bien des points communs avec la\\n                            vision biochimique. Toutes deux conviennent que les sensations agr\\u00e9ables\\n                            disparaissent aussi vite qu\\u2019elles naissent, et que les gens restent\\n                            insatisfaits tant qu\\u2019ils sont avides de sensations sans pour autant en\\n                            faire r\\u00e9ellement l\\u2019exp\\u00e9rience. Ce probl\\u00e8me admet cependant deux\\n                            solutions tr\\u00e8s diff\\u00e9rentes. La solution biochimique consiste \\u00e0 mettre au\\n                            point des produits et des traitements qui assureront aux \\u00eatres humains\\n                            un flux sans fin de sensations plaisantes, de sorte qu\\u2019ils n\\u2019en manquent\\n                            jamais. La suggestion du Bouddha consistait \\u00e0 r\\u00e9duire notre soif de\\n                            sensations agr\\u00e9ables et \\u00e0 ne pas les laisser dominer nos vies. \\u00c0 l\\u2019en\\n                            croire, nous pouvons exercer nos esprits \\u00e0 observer soigneusement\\n                            comment toutes nos sensations ne cessent de na\\u00eetre et de mourir. Quand\\n                            l\\u2019esprit apprend \\u00e0 voir nos sensations pour ce qu\\u2019elles sont \\u2013\\u00a0des\\n                            vibrations \\u00e9ph\\u00e9m\\u00e8res et d\\u00e9nu\\u00e9es de sens\\u00a0\\u2013, nous perdons tout int\\u00e9r\\u00eat \\u00e0\\n                            les poursuivre. Car \\u00e0 quoi bon poursuivre une chose qui dispara\\u00eet aussi\\n                            vite qu\\u2019elle appara\\u00eet\\u00a0?", "cluster": -1}, {"text": "Pour l\\u2019heure, l\\u2019humanit\\u00e9 s\\u2019int\\u00e9resse bien davantage \\u00e0 la solution\\n                            biochimique. Peu importe ce que disent les moines dans leurs grottes\\n                            himalayennes ou les philosophes dans leurs tours d\\u2019ivoire\\u00a0: pour\\n                            le mastodonte capitaliste, le bonheur est plaisir. Point barre. Chaque\\n                            ann\\u00e9e qui passe, notre tol\\u00e9rance des sensations d\\u00e9sagr\\u00e9ables diminue et\\n                            notre soif de sensations plaisantes augmente. La recherche scientifique\\n                            et l\\u2019activit\\u00e9 \\u00e9conomique sont l\\u2019une et l\\u2019autre attel\\u00e9es \\u00e0 cette\\n                            fin\\u00a0: chaque ann\\u00e9e produit son lot de meilleurs analg\\u00e9siques, de\\n                            nouveaux parfums de glaces, de matelas plus confortables et de jeux plus\\n                            addictifs sur nos smartphones pour que nous ne connaissions pas un seul\\n                            instant d\\u2019ennui en attendant le bus.", "cluster": 224}, {"text": "Tout cela ne suffit gu\\u00e8re, bien entendu.Homo sapiensne s\\u2019est pas\\n                            adapt\\u00e9 au gr\\u00e9 de l\\u2019\\u00e9volution pour \\u00e9prouver un plaisir constant\\u00a0:\\n                            si c\\u2019est n\\u00e9anmoins ce que cherche l\\u2019humanit\\u00e9, les glaces et les jeux\\n                            pour smartphones ne feront pas l\\u2019affaire. Il faudra changer notre\\n                            biochimie, et r\\u00e9organiser nos corps et nos esprits. C\\u2019est donc \\u00e0 cela\\n                            que nous travaillons. Qu\\u2019importe si cela est bien ou mal, mais il semble\\n                            que le deuxi\\u00e8me grand projet duXXIesi\\u00e8cle\\n                            \\u2013\\u00a0assurer le bonheur detous\\u00a0\\u2013 implique de manipulerHomo sapienspour qu\\u2019il puisse jouir d\\u2019un plaisir\\n                            \\u00e9ternel.", "cluster": 224}, {"text": "En recherchant la b\\u00e9atitude et l\\u2019immortalit\\u00e9, les \\u00eatres humains essaient\\n                            en fait de se hisser au rang de dieux\\u00a0: parce que ce sont des\\n                            qualit\\u00e9s divines, mais aussi parce que, pour triompher de la vieillesse\\n                            et de la mis\\u00e8re, ils devront d\\u2019abord acqu\\u00e9rir la ma\\u00eetrise divine de leur\\n                            substrat biologique. Si jamais nous poss\\u00e9dons le pouvoir d\\u2019\\u00e9radiquer de\\n                            notre syst\\u00e8me la mort et la douleur, ce m\\u00eame pouvoir suffira\\n                            probablement \\u00e0 organiser ledit syst\\u00e8me \\u00e0 notre guise, ou presque, et \\u00e0\\n                            manipuler de multiples fa\\u00e7ons nos organes, nos \\u00e9motions et notre\\n                            intelligence. Nous pourrions acqu\\u00e9rir la force d\\u2019Hercule, la sensualit\\u00e9\\n                            d\\u2019Aphrodite, la sagesse d\\u2019Ath\\u00e9na ou la folie de Dionysos, si c\\u2019est cela\\n                            qui nous chante. Jusqu\\u2019ici, accro\\u00eetre le pouvoir de l\\u2019homme reposait\\n                            exclusivement sur l\\u2019optimisation de nos outils externes. \\u00c0 l\\u2019avenir, il\\n                            d\\u00e9pendra davantage de l\\u2019optimisation du corps et de l\\u2019esprit humains,\\n                            voire d\\u2019une fusion directe avec nos outils.", "cluster": 259}, {"text": "Hisser les humains au rang des dieux peut se faire selon trois\\n                            directions\\u00a0: le g\\u00e9nie biologique, le g\\u00e9nie cyborg et le g\\u00e9nie des\\n                            \\u00eatres non organiques.", "cluster": -1}, {"text": "Le g\\u00e9nie biologique part de l\\u2019intuition que nous sommes loin de r\\u00e9aliser\\n                            pleinement le potentiel des corps organiques. Quatre\\u00a0milliards d\\u2019ann\\u00e9es\\n                            durant, la s\\u00e9lection naturelle a bricol\\u00e9 et bidouill\\u00e9 ces corps, en\\n                            sorte que nous sommes pass\\u00e9s successivement des amibes aux reptiles,\\n                            puis aux mammif\\u00e8res et aux Sapiens. Il n\\u2019y a aucune raison de penser que\\n                            Sapiens soit le dernier stade de cette \\u00e9volution. Des changements\\n                            relativement mineurs des g\\u00e8nes, des hormones et des neurones ont suffi \\u00e0\\n                            transformerHomo erectus\\u2013\\u00a0qui ne pouvait rien cr\\u00e9er de plus\\n                            impressionnant que des couteaux de silex \\u2013\\u00a0enHomo sapiens,\\n                            cr\\u00e9ateur de vaisseaux spatiaux et d\\u2019ordinateurs. Qui sait ce qui\\n                            pourrait r\\u00e9sulter de quelques changements suppl\\u00e9mentaires de notre ADN,\\n                            de notre syst\\u00e8me hormonal ou de la structure de notre cerveau\\u00a0? Le\\n                            g\\u00e9nie biologique ne va pas attendre patiemment que la magie de la\\n                            s\\u00e9lectionnaturelle op\\u00e8re. Les bio-ing\\u00e9nieurs vont\\n                            plut\\u00f4t se saisir du vieux corps de Sapiens et en r\\u00e9\\u00e9crire d\\u00e9lib\\u00e9r\\u00e9ment\\n                            le code g\\u00e9n\\u00e9tique, rec\\u00e2bler ses circuits c\\u00e9r\\u00e9braux, modifier son\\n                            \\u00e9quilibre biochimique, voire lui faire pousser des membres nouveaux. Ce\\n                            faisant, ils vont cr\\u00e9er desd\\u00e9it\\u00e9s, qui pourraient bien \\u00eatre\\n                            aussi diff\\u00e9rentes de nous, Sapiens, que nous le sommes d\\u2019Homo\\n                                erectus.", "cluster": -1}, {"text": "Le g\\u00e9nie cyborg ira plus loin, et fusionnera le corps organique avec des\\n                            appareils non organiques, tels que des mains bioniques, des yeux\\n                            artificiels ou des millions de nanorobots qui navigueront dans nos\\n                            vaisseaux sanguins, diagnostiqueront les probl\\u00e8mes et r\\u00e9pareront les\\n                            dommages. Les capacit\\u00e9s d\\u2019un tel cyborg pourraient \\u00eatre sans commune\\n                            mesure avec celles d\\u2019un corps organique. Par exemple, toutes les parties\\n                            d\\u2019un corps organique doivent \\u00eatre en contact les unes avec les autres\\n                            pour fonctionner. Si le cerveau d\\u2019un \\u00e9l\\u00e9phant est en Inde, ses yeux et\\n                            ses oreilles en Chine et ses pattes en Australie, c\\u2019est que cet \\u00e9l\\u00e9phant\\n                            est tr\\u00e8s vraisemblablement mort, et m\\u00eame si, en un sens myst\\u00e9rieux, il\\n                            est vivant, il ne saurait voir, entendre ou marcher. Un cyborg, en\\n                            revanche, pourrait exister en plusieurs endroits en m\\u00eame temps. Un\\n                            m\\u00e9decin cyborg pourrait accomplir des op\\u00e9rations en urgence \\u00e0 Tokyo, \\u00e0\\n                            Chicago et dans une station spatiale sur Mars sans avoir \\u00e0 quitter son\\n                            cabinet de Stockholm. Il lui suffira d\\u2019une connexion Internet rapide, et\\n                            de quelques paires d\\u2019yeux et de mains bioniques. \\u00c0 la r\\u00e9flexion,\\n                            pourquoi despaires? Pourquoi pas des quatuors\\u00a0? En\\n                            fait, m\\u00eame eux sont superflus. Pourquoi un m\\u00e9decin cyborg devrait-il\\n                            tenir un scalpel de chirurgien \\u00e0 la main, quand il pourrait connecter\\n                            directement son esprit \\u00e0 l\\u2019instrument\\u00a0?", "cluster": 140}, {"text": "On pourrait croire \\u00e0 de la science-fiction, mais c\\u2019est d\\u00e9j\\u00e0 une r\\u00e9alit\\u00e9.\\n                            Des singes ont r\\u00e9cemment appris \\u00e0 contr\\u00f4ler des mains et des pieds\\n                            bioniques d\\u00e9connect\\u00e9s de leur corps par le biais d\\u2019\\u00e9lectrodes implant\\u00e9es\\n                            dans leur cerveau. Par la seule force de la pens\\u00e9e, des patients\\n                            paralys\\u00e9s peuvent bouger des membres bioniques ou faire fonctionner des\\n                            ordinateurs. Si vous le souhaitez, vous pouvez d\\u2019ores et d\\u00e9j\\u00e0 utiliser\\n                            un casque \\u00ab\\u00a0qui lit dans les pens\\u00e9es\\u00a0\\u00bb pour contr\\u00f4ler \\u00e0\\n                            distance les appareils \\u00e9lectriques de votre maison. Le casque en\\n                            question ne n\\u00e9cessite pas d\\u2019implants c\\u00e9r\\u00e9braux. Il fonctionne en lisant\\n                            les signaux \\u00e9lectriques qui passent \\u00e0 travers votre cuir chevelu. Vous\\n                            voulez allumer la lumi\\u00e8rede la cuisine\\u00a0? Vous\\n                            n\\u2019avez qu\\u2019\\u00e0 porter le casque, imaginer un signal mental pr\\u00e9programm\\u00e9\\n                            (par exemple un mouvement de la main droite), et la lampe se met en\\n                            marche. Vous pouvez acheter des casques de ce type en ligne pour\\n                            400\\u00a0dollars seulement(44).", "cluster": -1}, {"text": "D\\u00e9but\\u00a02015, plusieurs centaines de travailleurs du foyer high-tech\\n                            Epicenter de Stockholm se sont fait implanter des puces dans les mains.\\n                            De la taille d\\u2019un grain de riz, elles stockent des informations de\\n                            s\\u00e9curit\\u00e9 personnalis\\u00e9es qui permettent aux employ\\u00e9s d\\u2019ouvrir des portes\\n                            ou de faire marcher des photocopieurs d\\u2019un simple geste de la main. Ils\\n                            esp\\u00e8rent bient\\u00f4t pouvoir effectuer des paiements de la m\\u00eame fa\\u00e7on.\\n                            \\u00ab\\u00a0Nous interagissons d\\u00e9j\\u00e0 tout le temps avec la technologie,\\n                            explique Hannes Sjoblad, l\\u2019un des inspirateurs de cette initiative.\\n                            Aujourd\\u2019hui, c\\u2019est un peu d\\u00e9sordonn\\u00e9\\u00a0: il nous faut des codes PIN\\n                            et des mots de passe. Ne serait-il pas plus facile d\\u2019utiliser simplement\\n                            la main(45)?\\u00a0\\u00bb", "cluster": -1}, {"text": "Pourtant, m\\u00eame le g\\u00e9nie cyborg est relativement conservateur, dans la\\n                            mesure o\\u00f9 il part du principe que les cerveaux organiques resteront les\\n                            centres de commandement et de contr\\u00f4le de la vie. Une approche plus\\n                            audacieuse consiste \\u00e0 se passer carr\\u00e9ment de parties organiques en\\n                            esp\\u00e9rant fabriquer des \\u00eatres enti\\u00e8rement non organiques. Les r\\u00e9seaux\\n                            neuronaux seront remplac\\u00e9s par un logiciel intelligent qui pourrait\\n                            surfer dans les mondes virtuel et non virtuel en \\u00e9chappant aux limites\\n                            de la chimie organique. Apr\\u00e8s quatre milliards d\\u2019ann\\u00e9es d\\u2019errance dans\\n                            le royaume des compos\\u00e9s organiques, la vie fera irruption dans\\n                            l\\u2019immensit\\u00e9 du champ inorganique et prendra des formes qu\\u2019on ne saurait\\n                            envisager m\\u00eame dans nos r\\u00eaves les plus fous. Apr\\u00e8s tout, nos r\\u00eaves les\\n                            plus fous sont encore le produit de la chimie organique.", "cluster": 140}, {"text": "Se lib\\u00e9rer du domaine organique pourrait aussi permettre \\u00e0 la vie de se\\n                            lib\\u00e9rer finalement de la Terre. Depuis quatre\\u00a0milliards d\\u2019ann\\u00e9es, si la\\n                            vie est demeur\\u00e9e confin\\u00e9e \\u00e0 cette minuscule graine de plan\\u00e8te, c\\u2019est\\n                            parce que la s\\u00e9lection naturelle a rendu tous les organismes totalement\\n                            d\\u00e9pendants des conditions uniques en leur genre de ce rocher volant.\\n                            M\\u00eame les bact\\u00e9ries les plus coriaces ne sauraient survivre sur Mars. En\\n                            revanche, une intelligence artificielle non organique trouvera bien plus\\n                            facile de coloniser des plan\\u00e8tes \\u00e9trang\\u00e8res. Le remplacement de la vie\\n                            organique par des \\u00eatres non organiques porte en germe l\\u2019\\u00e9mergenced\\u2019un futur empire galactique dirig\\u00e9 par les avatars de\\n                            Mr.\\u00a0Data, l\\u2019andro\\u00efde deStar Trek, plut\\u00f4t que par ceux du\\n                            capitaine Kirk.", "cluster": 140}, {"text": "Nous ne savons pas o\\u00f9 ces chemins pourraient nous conduire, ni \\u00e0 quoi\\n                            ressembleraient nos descendants divins. Pr\\u00e9voir l\\u2019avenir n\\u2019a jamais \\u00e9t\\u00e9\\n                            facile, et les biotechnologies r\\u00e9volutionnaires rendent la chose encore\\n                            plus difficile. Car si ardu qu\\u2019il soit de pr\\u00e9dire l\\u2019impact des nouvelles\\n                            technologies dans des domaines comme les transports, les communications\\n                            et l\\u2019\\u00e9nergie, les technologies visant l\\u2019augmentation du potentiel humain\\n                            sont un d\\u00e9fi d\\u2019une tout autre nature. Puisque, par d\\u00e9finition, elles\\n                            sont capables de transformer les esprits et les d\\u00e9sirs des hommes, les\\n                            gens d\\u2019aujourd\\u2019hui, avec leurs esprits et leurs d\\u00e9sirs d\\u2019aujourd\\u2019hui, ne\\n                            sauraient en sonder les implications.", "cluster": 196}, {"text": "Au fil des mill\\u00e9naires, l\\u2019histoire a \\u00e9t\\u00e9 riche en bouleversements\\n                            techniques, \\u00e9conomiques, sociaux et politiques. La seule constante a \\u00e9t\\u00e9\\n                            l\\u2019humanit\\u00e9 elle-m\\u00eame. Nos outils et institutions sont tr\\u00e8s diff\\u00e9rents de\\n                            ceux des temps bibliques, mais les structures profondes de l\\u2019esprit\\n                            humain restent les m\\u00eames. C\\u2019est bien pourquoi nous pouvons encore nous\\n                            reconna\\u00eetre dans les pages de la Bible, dans les \\u00e9crits de Confucius ou\\n                            dans les trag\\u00e9dies de Sophocle et d\\u2019Euripide. Ces classiques sont\\n                            l\\u2019\\u0153uvre d\\u2019hommes exactement pareils \\u00e0 nous, d\\u2019o\\u00f9 notre sentiment qu\\u2019ils\\n                            parlent de nous. Dans le th\\u00e9\\u00e2tre moderne, \\u0152dipe, Hamlet et Othello\\n                            peuvent bien porter des jeans et des T-shirts et avoir des comptes\\n                            Facebook, leurs dilemmes sont les m\\u00eames que dans la pi\\u00e8ce originale.", "cluster": -1}, {"text": "Toutefois, le jour o\\u00f9 la technologie nous permettra de r\\u00e9agencer l\\u2019esprit\\n                            humain,Homo sapiensdispara\\u00eetra et l\\u2019histoire humaine touchera \\u00e0\\n                            sa fin. S\\u2019amorcera alors un processus d\\u2019un genre enti\\u00e8rement nouveau,\\n                            que ni vous ni moi ne saurions comprendre. Beaucoup de chercheurs\\n                            essaient de pr\\u00e9dire \\u00e0 quoi le monde ressemblera en 2100 ou 2200. Ils\\n                            perdent leur temps. Toute pr\\u00e9diction digne de ce nom doit prendre en\\n                            compte la capacit\\u00e9 de r\\u00e9agencer l\\u2019esprit humain \\u2013\\u00a0ce qui est impossible.\\n                            Il y a nombre de r\\u00e9ponses empreintes de sagesse \\u00e0 la question\\u00a0:\\n                            \\u00ab\\u00a0Que feraient de la biotechnologie des gens poss\\u00e9dant des esprits\\n                            commele n\\u00f4tre\\u00a0?\\u00a0\\u00bb Mais aucune bonne r\\u00e9ponse\\n                            \\u00e0 cette autre question\\u00a0: \\u00ab\\u00a0Que feraient de la biotechnologie\\n                            des gens poss\\u00e9dant une sorte d\\u2019espritdiff\\u00e9rentedu\\n                            n\\u00f4tre\\u00a0?\\u00a0\\u00bb La seule chose qu\\u2019on puisse dire, c\\u2019est que des\\n                            gens semblables \\u00e0 nous utiliseront probablement la biotechnologie pour\\n                            r\\u00e9organiser leurs esprits, mais nos esprits d\\u2019aujourd\\u2019hui ne sauraient\\n                            penser ce qui pourrait arriver ensuite.", "cluster": 196}, {"text": "Bien que les d\\u00e9tails soient donc obscurs, la direction g\\u00e9n\\u00e9rale de\\n                            l\\u2019histoire ne fait pas de doute. AuXXIesi\\u00e8cle,\\n                            l\\u2019humanit\\u00e9 se fixera comme troisi\\u00e8me grand projet d\\u2019acqu\\u00e9rir des\\n                            pouvoirs divins de cr\\u00e9ation et de destruction, et de hisserHomo\\n                                sapiensau rang d\\u2019Homo deus. Ce troisi\\u00e8me projet subsume\\n                            \\u00e0 l\\u2019\\u00e9vidence les deux premiers et s\\u2019en nourrit. Nous d\\u00e9sirons avant tout\\n                            \\u00eatre capables de r\\u00e9agencer nos corps et nos esprits pour \\u00e9chapper \\u00e0 la\\n                            vieillesse, \\u00e0 la mort et \\u00e0 la mis\\u00e8re, mais, ce but atteint, qui sait \\u00e0\\n                            quoi nous emploierons ensuite cette capacit\\u00e9\\u00a0? Il n\\u2019est donc pas\\n                            extravagant de penser que le nouvel ordre du jour humain ne consiste au\\n                            fond qu\\u2019en un seul projet, avec de multiples branches\\u00a0: atteindre\\n                            la divinit\\u00e9.", "cluster": 258}, {"text": "Si cela semble peu scientifique, voire carr\\u00e9ment excentrique, c\\u2019est que\\n                            les gens se m\\u00e9prennent souvent sur le sens de cette nature divine. La\\n                            divinit\\u00e9 n\\u2019est pas une vague qualit\\u00e9 m\\u00e9taphysique. Et ce n\\u2019est pas la\\n                            m\\u00eame chose que l\\u2019omnipotence. Quand on parle de hisser les humains au\\n                            rang de dieux, il faut penser plus aux dieux grecs ou aux d\\u00e9vas hindous\\n                            qu\\u2019au p\\u00e8re tout-puissant du ciel biblique. Nos descendants auraient\\n                            encore leurs faiblesses, leurs bizarreries et leurs limites, tout comme\\n                            Zeus et Indra avaient les leurs. Mais ils pourraient aimer, ha\\u00efr, cr\\u00e9er\\n                            et d\\u00e9truire \\u00e0 bien plus grande \\u00e9chelle que nous.", "cluster": 251}, {"text": "Tout au long de l\\u2019histoire, on a pr\\u00eat\\u00e9 \\u00e0 la plupart des dieux non pas\\n                            l\\u2019omnipotence, mais des super-capacit\\u00e9s assez sp\\u00e9cifiques, comme celle\\n                            de concevoir et de cr\\u00e9er des \\u00eatres vivants, de transformer leurs corps,\\n                            de contr\\u00f4ler leur environnement et le temps\\u00a0; de lire dans les\\n                            esprits et de communiquer \\u00e0 distance\\u00a0; de voyager \\u00e0 tr\\u00e8s grande\\n                            vitesse\\u00a0; et, bien entendu, d\\u2019\\u00e9chapper \\u00e0 la mort et de vivre\\n                            \\u00e9ternellement. Les hommes s\\u2019emploient \\u00e0 acqu\\u00e9rir toutes ces capacit\\u00e9s,\\n                            voire plus.", "cluster": 251}, {"text": "Certaines capacit\\u00e9s traditionnelles, r\\u00e9put\\u00e9es divines des mill\\u00e9naires\\n                            durant, sont devenues aujourd\\u2019hui si ordinaires que c\\u2019est \\u00e0 peine si\\n                            nous y pensons. L\\u2019homme moyen se d\\u00e9place et communiqueaujourd\\u2019hui bien plus ais\\u00e9ment que les dieux grecs, hindous ou\\n                            africains d\\u2019antan. Par exemple, selon les Igbo du Nigeria, Chukwu, le\\n                            dieu cr\\u00e9ateur, voulait initialement rendre les hommes immortels. Il\\n                            envoya un chien leur expliquer qu\\u2019\\u00e0 la mort de l\\u2019un d\\u2019entre eux ils\\n                            devaient saupoudrer le cadavre de cendres, et que le corps reviendrait\\n                            alors \\u00e0 la vie. Malheureusement, le chien \\u00e9tait fatigu\\u00e9 et a tra\\u00een\\u00e9.\\n                            Impatient, Chukwu a alors envoy\\u00e9 un mouton, le priant de porter sans\\n                            d\\u00e9lai cet important message. H\\u00e9las, quand le mouton essouffl\\u00e9 parvint \\u00e0\\n                            destination, il s\\u2019embrouilla et dit aux hommes d\\u2019enterrer leurs morts,\\n                            ce qui eut pour effet de rendre la mort irr\\u00e9versible. Voil\\u00e0 pourquoi,\\n                            aujourd\\u2019hui encore, les hommes doivent mourir. Si seulement Chukwu avait\\n                            eu un compte Twitter au lieu de s\\u2019en remettre \\u00e0 un chien tra\\u00eenard et \\u00e0\\n                            un imb\\u00e9cile de mouton pour transmettre ses messages\\u00a0!", "cluster": -1}, {"text": "Dans les soci\\u00e9t\\u00e9s agricoles anciennes, beaucoup de religions montraient\\n                            \\u00e9tonnamment peu d\\u2019int\\u00e9r\\u00eat pour les questions m\\u00e9taphysiques et l\\u2019au-del\\u00e0.\\n                            Elles se focalisaient plut\\u00f4t sur la question tr\\u00e8s prosa\\u00efque de\\n                            l\\u2019augmentation de la production agricole. Ainsi l\\u2019Ancien Testament ne\\n                            promet-iljamaisni r\\u00e9compense ni ch\\u00e2timent apr\\u00e8s la mort, mais\\n                            dit plut\\u00f4t au peuple d\\u2019Isra\\u00ebl\\u00a0: \\u00ab\\u00a0Si vous ob\\u00e9issez \\u00e0 mes\\n                            commandements que je vous prescris aujourd\\u2019hui, si vous aimez l\\u2019\\u00c9ternel,\\n                            votre Dieu [\\u2026], je donnerai \\u00e0 votre pays la pluie en son temps [\\u2026], et\\n                            tu recueilleras ton bl\\u00e9, ton mo\\u00fbt et ton huile\\u00a0; je mettrai aussi\\n                            dans tes champs de l\\u2019herbe pour ton b\\u00e9tail, et tu mangeras et te\\n                            rassasieras. Gardez-vous de laisser s\\u00e9duire votre c\\u0153ur, de vous\\n                            d\\u00e9tourner, de servir d\\u2019autres dieux et de vous prosterner devant eux. La\\n                            col\\u00e8re de l\\u2019\\u00c9ternel s\\u2019enflammerait alors contre vous\\u00a0; il\\n                            fermerait les cieux, et il n\\u2019y aurait point de pluie\\u00a0; la terre ne\\n                            donnerait plus ses produits, et vous p\\u00e9ririez promptement dans le bon\\n                            pays que l\\u2019\\u00c9ternel vous donne\\u00a0\\u00bb (Deut\\u00e9ronome 11,13-17(46)). Les hommes de science font aujourd\\u2019hui beaucoup\\n                            mieux que le Dieu de l\\u2019Ancien Testament. Gr\\u00e2ce aux engrais artificiels,\\n                            aux insecticides industriels et aux organismes g\\u00e9n\\u00e9tiquement modifi\\u00e9s,\\n                            la production agricole d\\u00e9passe maintenant largement les attentes que les\\n                            paysans d\\u2019antan avaient \\u00e0 l\\u2019\\u00e9gard de leurs dieux, m\\u00eame dans leurs r\\u00eaves\\n                            les plus fous. Et l\\u2019\\u00c9tat ass\\u00e9ch\\u00e9 d\\u2019Isra\\u00ebl ne craint plus qu\\u2019un dieu en\\n                            courroux ferme les cieux et arr\\u00eate la pluie, car les Isra\\u00e9liens ont\\n                            derni\\u00e8rementconstruit sur les c\\u00f4tes de la M\\u00e9diterran\\u00e9e\\n                            une immense usine de d\\u00e9salinisation et tirent d\\u00e9sormais leur eau potable\\n                            de la mer.", "cluster": 100}, {"text": "Jusque-l\\u00e0, nous avons rivalis\\u00e9 avec les dieux d\\u2019antan en cr\\u00e9ant des\\n                            outils toujours plus performants. Dans un avenir pas si lointain, nous\\n                            pourrions bien cr\\u00e9er des surhommes qui surpasseraient les dieux antiques\\n                            gr\\u00e2ce \\u00e0 leurs outils, mais aussi \\u00e0 leurs facult\\u00e9s corporelles et\\n                            mentales. Quand nous en serons l\\u00e0, cependant, la divinit\\u00e9 deviendra\\n                            aussi banale que le cyberespace\\u00a0: une merveille des merveilles que\\n                            nous tenons pour acquise.", "cluster": 251}, {"text": "Nous pouvons \\u00eatre certains que les hommes feront cette ench\\u00e8re, parce que\\n                            les humains ont maintes raisons de d\\u00e9sirer une telle promotion, et de\\n                            multiples fa\\u00e7ons d\\u2019y parvenir. M\\u00eame si un chemin qui semblait prometteur\\n                            finit en impasse, d\\u2019autres routes demeureront ouvertes. Par exemple,\\n                            nous d\\u00e9couvrirons peut-\\u00eatre que le g\\u00e9nome humain est bien trop compliqu\\u00e9\\n                            pour se pr\\u00eater \\u00e0 une manipulation s\\u00e9rieuse, mais cela n\\u2019emp\\u00eachera pas le\\n                            d\\u00e9veloppement d\\u2019interfaces cerveau-ordinateurs, de nanorobots ou de\\n                            l\\u2019intelligence artificielle.", "cluster": -1}, {"text": "Mais pas de panique\\u00a0! Du moins, pas tout de suite. L\\u2019ascension de\\n                            Sapiens sera un processus historique progressif plut\\u00f4t qu\\u2019une apocalypse\\n                            hollywoodienne. Une r\\u00e9volte des robots n\\u2019exterminera pasHomo\\n                                sapiens. Celui-ci s\\u2019\\u00e9l\\u00e8vera sans doute pas \\u00e0 pas, fusionnant en\\n                            cours de route avec des robots et des ordinateurs, et, un jour, nos\\n                            descendants, se retournant sur le pass\\u00e9, s\\u2019apercevront qu\\u2019ils ne sont\\n                            plus le genre d\\u2019animal qui a \\u00e9crit la Bible, construit la Grande\\n                            Muraille de Chine et ri des pitreries de Charlie Chaplin. Cela ne se\\n                            produira pas en un jour, ni en une ann\\u00e9e. Cela commence aujourd\\u2019hui, \\u00e0\\n                            travers une multitude d\\u2019actions banales. Chaque jour, des millions de\\n                            gens d\\u00e9cident de confier \\u00e0 leur smartphone un plus large contr\\u00f4le sur\\n                            leur vie, ou d\\u2019essayer un nouvel antid\\u00e9presseur plus efficace. Dans la\\n                            qu\\u00eate de la sant\\u00e9, du bonheur et du pouvoir, les hommes se mettront \\u00e0\\n                            modifier peu \\u00e0 peu un de leurs traits, puis un autre, et encore un\\n                            autre, jusqu\\u2019\\u00e0 ce qu\\u2019ils ne soient plus humains.", "cluster": -1}, {"text": "S\\u2019IL VOUS PLA\\u00ceT, Y A-T-IL QUELQU\\u2019UN POUR DONNER UN COUP DE\\n                                FREIN?", "cluster": -1}, {"text": "Rejetant les explications rationnelles, beaucoup de gens paniquent quand\\n                            on leur parle de ces possibilit\\u00e9s. Ils sont heureux de suivre les\\n                            conseils de leur smartphone ou de prendre les m\\u00e9dicaments prescrits par\\n                            leur m\\u00e9decin, mais qu\\u2019on leur parle de surhommes am\\u00e9lior\\u00e9s et ils\\n                            r\\u00e9pondent\\u00a0: \\u00ab\\u00a0J\\u2019esp\\u00e8re que je serai mort avant que \\u00e7a\\n                            n\\u2019arrive.\\u00a0\\u00bb Ce qu\\u2019elle craint le plus dans la vieillesse, me\\n                            disait un jour une amie, c\\u2019est de ne plus avoir aucun poids, de se\\n                            transformer en vieille femme nostalgique incapable de comprendre le\\n                            monde qui l\\u2019entoure ou d\\u2019y contribuer vraiment. C\\u2019est ce que nous\\n                            croyons collectivement, en tant qu\\u2019esp\\u00e8ce, quand nous entendons parler\\n                            de surhommes. Nous avons le sentiment que, dans un monde de ce genre,\\n                            notre identit\\u00e9, nos r\\u00eaves et m\\u00eame nos peurs perdront toute importance,\\n                            et que nous n\\u2019aurons plus rien \\u00e0 apporter. Qui que vous soyez\\n                            aujourd\\u2019hui \\u2013\\u00a0fervent joueur de cricket hindou ou journaliste en herbe\\n                            lesbienne\\u00a0\\u2013, dans un monde am\\u00e9lior\\u00e9 vous vous sentirez tel un chasseur\\n                            Neandertal \\u00e0 Wall Street. Vous ne serez pas \\u00e0 votre place.", "cluster": -1}, {"text": "Les Neandertal n\\u2019avaient pas \\u00e0 s\\u2019inqui\\u00e9ter du Nasdaq, dont plusieurs\\n                            dizaines de milliers d\\u2019ann\\u00e9es les prot\\u00e9geaient. De nos jours, cependant,\\n                            notre monde de sens pourrait s\\u2019effondrer en quelques d\\u00e9cennies. On ne\\n                            saurait compter sur la mort pour nous \\u00e9pargner de perdre tout int\\u00e9r\\u00eat.\\n                            M\\u00eame si les dieux ne d\\u00e9ambulent pas dans nos rues en 2100, l\\u2019effort\\n                            d\\u00e9ploy\\u00e9 pour am\\u00e9liorerHomo sapiensest susceptible de rendre le\\n                            monde m\\u00e9connaissable au cours de ce si\\u00e8cle. La recherche scientifique et\\n                            les progr\\u00e8s techniques vont bien plus vite que ce que la plupart d\\u2019entre\\n                            nous pouvons saisir.", "cluster": -1}, {"text": "Si vous parlez avec des experts, beaucoup vous diront que nous sommes\\n                            encore tr\\u00e8s loin des b\\u00e9b\\u00e9s g\\u00e9n\\u00e9tiquement modifi\\u00e9s ou d\\u2019une intelligence\\n                            artificielle au niveau de l\\u2019homme. Mais la pens\\u00e9e de la plupart des\\n                            experts est r\\u00e9gl\\u00e9e sur l\\u2019\\u00e9chelle temporelle des bourses et des postes.\\n                            Aussi \\u00ab\\u00a0tr\\u00e8s loin\\u00a0\\u00bb peut-il signifier vingt\\u00a0ans et\\n                            \\u00ab\\u00a0jamais\\u00a0\\u00bb pas plus de cinquante\\u00a0ans.", "cluster": -1}, {"text": "Je me souviens encore du jour o\\u00f9 j\\u2019ai d\\u00e9couvert l\\u2019Internet. C\\u2019\\u00e9tait en\\n                            1993. J\\u2019\\u00e9tais au lyc\\u00e9e. Avec deux copains, je suis all\\u00e9 voir notre amiIdo, aujourd\\u2019hui informaticien. Nous voulions jouer\\n                            au ping-pong. Ido \\u00e9tait d\\u00e9j\\u00e0 un passionn\\u00e9 d\\u2019ordinateur, et avant de\\n                            commencer \\u00e0 jouer il a tenu \\u00e0 nous montrer la derni\\u00e8re merveille. Il a\\n                            branch\\u00e9 le c\\u00e2ble du t\\u00e9l\\u00e9phone \\u00e0 son ordinateur et appuy\\u00e9 sur des\\n                            touches. Pendant une minute, nous avons entendu des grincements, des\\n                            sons per\\u00e7ants et des vrombissements, puis le silence. \\u00c9chec. Nous avons\\n                            ronchonn\\u00e9, mais Ido a r\\u00e9essay\\u00e9. Encore et encore. Il a fini par pousser\\n                            un cri de joie et annonc\\u00e9 qu\\u2019il avait r\\u00e9ussi \\u00e0 connecter son appareil \\u00e0\\n                            l\\u2019ordinateur central de l\\u2019universit\\u00e9 voisine. \\u00ab\\u00a0Et qu\\u2019est-ce qu\\u2019il\\n                            y a sur l\\u2019ordinateur central\\u00a0?\\u00a0\\u00bb avons-nous demand\\u00e9.\\n                            \\u00ab\\u00a0Ben, a-t-il reconnu, pour l\\u2019instant, rien. Mais on pourrait y\\n                            mettre des tas de choses. \\u2013\\u00a0Comme quoi\\u00a0? \\u2013\\u00a0Je ne sais pas\\u2026 Toutes\\n                            sortes de choses.\\u00a0\\u00bb \\u00c7a ne paraissait pas tr\\u00e8s prometteur. Nous\\n                            sommes all\\u00e9s jouer au ping-pong. Les semaines suivantes, nous avons\\n                            profit\\u00e9 de ce nouveau passe-temps, tout en nous moquant de l\\u2019id\\u00e9e\\n                            ridicule d\\u2019Ido. Cela se passait il y a vingt-cinq\\u00a0ans (\\u00e0 l\\u2019heure o\\u00f9\\n                            j\\u2019\\u00e9cris). Qui sait ce qui arrivera d\\u2019ici vingt-cinq\\u00a0ans\\u00a0?", "cluster": -1}, {"text": "C\\u2019est pourquoi un nombre toujours plus grand d\\u2019individus,\\n                            d\\u2019organisations, d\\u2019entreprises et de gouvernements prennent tr\\u00e8s au\\n                            s\\u00e9rieux la qu\\u00eate de l\\u2019immortalit\\u00e9, du bonheur et de pouvoirs divins.\\n                            Compagnies d\\u2019assurance, fonds de pension, syst\\u00e8mes de sant\\u00e9 et\\n                            minist\\u00e8res des Finances sont d\\u00e9j\\u00e0 constern\\u00e9s par cette progression de\\n                            l\\u2019esp\\u00e9rance de vie. Les gens vivent beaucoup plus longtemps que pr\\u00e9vu,\\n                            et il n\\u2019y a pas assez d\\u2019argent pour payer les retraites et les soins\\n                            m\\u00e9dicaux. Alors que le cap des soixante-dix\\u00a0ans menace de devenir la\\n                            nouvelle quarantaine, les experts recommandent de repousser l\\u2019\\u00e2ge de la\\n                            retraite et de restructurer tout le march\\u00e9 de l\\u2019emploi.", "cluster": -1}, {"text": "Quand les gens comprennent \\u00e0 quelle vitesse nous nous pr\\u00e9cipitons vers le\\n                            grand inconnu, et qu\\u2019ils ne peuvent m\\u00eame pas compter sur la mort pour\\n                            les en pr\\u00e9server, leur r\\u00e9action est d\\u2019esp\\u00e9rer que quelqu\\u2019un donne un\\n                            coup de frein et nous fasse ralentir. Or, pour diverses raisons, il est\\n                            impossible de freiner.", "cluster": 38}, {"text": "Pour commencer, personne ne sait o\\u00f9 sont les freins. Certains experts\\n                            suivent ce qui se passe dans un domaine, comme l\\u2019intelligence\\n                            artificielle, les nanotechnologies et le \\u00ab\\u00a0Big Data\\u00a0\\u00bb ou la\\n                            g\\u00e9n\\u00e9tique, mais personne n\\u2019est expert en tout. Personne n\\u2019est donc\\n                            capable de relier tous les points et d\\u2019avoir une vue d\\u2019ensemble. Les\\n                            diff\\u00e9rents domainess\\u2019influencent mutuellement avec une\\n                            telle complexit\\u00e9 que m\\u00eame les meilleurs esprits ne sauraient deviner en\\n                            quoi les perc\\u00e9es de l\\u2019intelligence artificielle pourraient avoir un\\n                            impact sur les nanotechnologies, et inversement. Personne ne peut\\n                            assimiler toutes les d\\u00e9couvertes scientifiques les plus r\\u00e9centes,\\n                            personne ne peut pr\\u00e9dire de quoi l\\u2019\\u00e9conomie mondiale sera faite dans\\n                            dix\\u00a0ans, et personne n\\u2019a la moindre id\\u00e9e de ce vers quoi nous nous\\n                            dirigeons avec tant de h\\u00e2te. Personne ne comprenant plus le syst\\u00e8me, nul\\n                            ne peut l\\u2019arr\\u00eater.", "cluster": 38}, {"text": "Par ailleurs, si nous parvenons tant bien que mal \\u00e0 freiner, notre\\n                            \\u00e9conomie s\\u2019effondrera en m\\u00eame temps que notre soci\\u00e9t\\u00e9. Comme on le verra\\n                            plus loin, l\\u2019\\u00e9conomie moderne a besoin d\\u2019une croissance constante et\\n                            illimit\\u00e9e pour survivre. Si jamais la croissance s\\u2019arr\\u00eate, l\\u2019\\u00e9conomie ne\\n                            s\\u2019installera pas dans un \\u00e9quilibre douillet, elle se disloquera. C\\u2019est\\n                            bien pour cela que le capitalisme nous encourage \\u00e0 rechercher\\n                            l\\u2019immortalit\\u00e9, le bonheur et la divinit\\u00e9. Le nombre de chaussures que\\n                            nous pouvons porter, de voitures que nous pouvons conduire ou de\\n                            vacances au ski que nous pouvons prendre est limit\\u00e9. Une \\u00e9conomie fond\\u00e9e\\n                            sur une croissance \\u00e9ternelle a besoin de projets sans fin, comme la\\n                            qu\\u00eate de l\\u2019immortalit\\u00e9, du bonheur supr\\u00eame et de la divinit\\u00e9.", "cluster": 147}, {"text": "Alors, pourquoi ne pas choisir le bonheur et l\\u2019immortalit\\u00e9, et laisser au\\n                            moins de c\\u00f4t\\u00e9 la qu\\u00eate effrayante de forces surhumaines\\u00a0? Parce\\n                            que cette qu\\u00eate est inextricablement li\\u00e9e aux deux autres. Les jambes\\n                            bioniques qui permettent \\u00e0 un parapl\\u00e9gique de marcher de nouveau peuvent\\n                            aussi perfectionner les gens en bonne sant\\u00e9. Le m\\u00eame traitement qui\\n                            permet d\\u2019enrayer la perte de m\\u00e9moire chez les personnes \\u00e2g\\u00e9es pourrait\\n                            renforcer la m\\u00e9moire des jeunes.", "cluster": 259}, {"text": "Il n\\u2019existe pas de diff\\u00e9rence claire entre la gu\\u00e9rison et l\\u2019am\\u00e9lioration.\\n                            Si la m\\u00e9decine commence presque toujours par emp\\u00eacher les gens de tomber\\n                            au-dessous de la norme, elle peut employer les m\\u00eames moyens et le m\\u00eame\\n                            savoir-faire pour surpasser la norme. Le Viagra a d\\u2019abord \\u00e9t\\u00e9 un\\n                            m\\u00e9dicament traitant les probl\\u00e8mes de tension. \\u00c0 la surprise de Pfizer,\\n                            et pour son plus grand plaisir, il est apparu que le Viagra pouvait\\n                            aussi rem\\u00e9dier \\u00e0 l\\u2019impuissance. Il a permis \\u00e0 des millions d\\u2019hommes de\\n                            retrouver des capacit\\u00e9s sexuelles normales, mais, assez vite, des hommes\\n                            qui n\\u2019avaient pas de probl\\u00e8mes d\\u2019impuissance se sont mis \\u00e0 employer la\\n                            m\\u00eame pilule pour surpasser lanorme, et acqu\\u00e9rir une\\n                            puissance sexuelle qu\\u2019ils n\\u2019avaient jamais connue auparavant(47).", "cluster": 85}, {"text": "Ce qu\\u2019il advient de tel ou tel m\\u00e9dicament en particulier peut aussi se\\n                            produire dans des champs entiers de la m\\u00e9decine. La chirurgie esth\\u00e9tique\\n                            moderne est n\\u00e9e au cours de la Premi\\u00e8re Guerre mondiale, quand Harold\\n                            Gillies a commenc\\u00e9 \\u00e0 traiter les gueules cass\\u00e9es de l\\u2019h\\u00f4pital militaire\\n                                d\\u2019Aldershot(48). Une fois la guerre finie, des\\n                            chirurgiens d\\u00e9couvrirent que les m\\u00eames techniques pouvaient servir \\u00e0\\n                            transformer des nez parfaitement sains, mais laids, en plus beaux\\n                            sp\\u00e9cimens. Bien que la chirurgie plastique ait continu\\u00e9 d\\u2019aider les\\n                            malades et les bless\\u00e9s, elle s\\u2019est de plus en plus souci\\u00e9e d\\u2019am\\u00e9liorer\\n                            les gens en bonne sant\\u00e9. De nos jours, les chirurgiens esth\\u00e9tiques\\n                            gagnent des millions dans des cliniques priv\\u00e9es dont le but explicite,\\n                            et unique, est d\\u2019am\\u00e9liorer les gens en bonne sant\\u00e9 et d\\u2019embellir les\\n                                riches(49).", "cluster": -1}, {"text": "Le m\\u00eame sc\\u00e9nario pourrait se produire avec le g\\u00e9nie g\\u00e9n\\u00e9tique. Si un\\n                            milliardaire d\\u00e9clarait ouvertement son intention de fabriquer une\\n                            prog\\u00e9niture super-intelligente, il provoquerait un toll\\u00e9 g\\u00e9n\\u00e9ral. Mais\\n                            cela ne se passera pas ainsi. Plus probablement nous engagerons-nous sur\\n                            une pente glissante. Tout commence par les parents, dont le profil\\n                            g\\u00e9n\\u00e9tique fait courir \\u00e0 leurs enfants de gros risques de maladies\\n                            g\\u00e9n\\u00e9tiques mortelles. Ils recourent donc \\u00e0 la f\\u00e9condationin\\n                                vitroet testent l\\u2019ADN de l\\u2019\\u0153uf f\\u00e9cond\\u00e9. Si tout est en ordre,\\n                            parfait. Mais si le test ADN d\\u00e9couvre les mutations redout\\u00e9es, l\\u2019embryon\\n                            est d\\u00e9truit.", "cluster": 252}, {"text": "Pourquoi prendre un risque en ne f\\u00e9condant qu\\u2019un seul \\u0153uf\\u00a0? Mieux\\n                            vaut en f\\u00e9conder plusieurs\\u00a0: si trois ou quatre laissaient \\u00e0\\n                            d\\u00e9sirer, il y aurait au moins un bon embryon. Cette proc\\u00e9dure de\\n                            s\\u00e9lectionin vitrodevenant acceptable et bon march\\u00e9, son usage\\n                            peut se r\\u00e9pandre. Les mutations sont un risque omnipr\\u00e9sent. Tous les\\n                            individus portent dans leur ADN des mutations nocives et des all\\u00e8les qui\\n                            sont loin d\\u2019\\u00eatre optimaux. La reproduction sexuelle est une loterie. Une\\n                            anecdote c\\u00e9l\\u00e8bre, probablement apocryphe, rapporte la rencontre en 1923\\n                            du prix Nobel de litt\\u00e9rature Anatole France et d\\u2019Isadora Duncan, la\\n                            belle et talentueuse danseuse. Discutant du mouvement eug\\u00e9niste alors en\\n                            vogue, Duncan observa\\u00a0: \\u00ab\\u00a0Imaginez un peu un enfant qui\\n                            aurait ma beaut\\u00e9 et votre intelligence\\u00a0!\\u00a0\\u00bb Et France de\\n                            r\\u00e9pondre\\u00a0: \\u00ab\\u00a0Oui, mais imaginez un enfant qui aitmabeaut\\u00e9 etvotreintelligence\\u00a0!\\u00a0\\u00bb Alors, pourquoi\\n                            ne pastruquer la loterie \\u2013\\u00a0f\\u00e9conder plusieurs \\u0153ufs et\\n                            choisir celui qui offre la meilleure combinaison\\u00a0? Puisque la\\n                            recherche sur les cellules souches permet de cr\\u00e9er une offre illimit\\u00e9e\\n                            d\\u2019embryons humains, libre \\u00e0 vous de choisir votre b\\u00e9b\\u00e9 optimal parmi des\\n                            centaines de candidats, tous porteurs de votre ADN et tous parfaitement\\n                            naturels, sans qu\\u2019aucun ne n\\u00e9cessite le moindre g\\u00e9nie g\\u00e9n\\u00e9tique\\n                            futuriste. R\\u00e9p\\u00e9tez cette proc\\u00e9dure pendant quelques g\\u00e9n\\u00e9rations, et vous\\n                            pourrez ais\\u00e9ment finir avec des surhommes (ou une dystopie glauque).", "cluster": 252}, {"text": "Et si m\\u00eame apr\\u00e8s avoir f\\u00e9cond\\u00e9 plusieurs \\u0153ufs vous constatez qu\\u2019ils\\n                            contiennent tous des mutations mortelles\\u00a0? Faut-il d\\u00e9truire tous\\n                            les embryons\\u00a0? Pourquoi ne pas remplacer plut\\u00f4t les g\\u00e8nes\\n                            probl\\u00e9matiques\\u00a0? Une avanc\\u00e9e scientifique a \\u00e9t\\u00e9 r\\u00e9alis\\u00e9e\\n                            concernant l\\u2019ADN mitochondrial. Les mitochondries sont de minuscules\\n                            organites au sein des cellules humaines produisant l\\u2019\\u00e9nergie utilis\\u00e9e\\n                            par celles-ci. Elles poss\\u00e8dent leur propre ensemble de g\\u00e8nes,\\n                            enti\\u00e8rement distinct de l\\u2019ADN du noyau de la cellule. Une d\\u00e9ficience de\\n                            l\\u2019ADN mitochondrial se traduit par diverses maladies handicapantes,\\n                            voire mortelles. Par la technique de la f\\u00e9condationin vitro, il\\n                            est techniquement possible de triompher des maladies g\\u00e9n\\u00e9tiques\\n                            mitochondriales en cr\\u00e9ant un \\u00ab\\u00a0b\\u00e9b\\u00e9 \\u00e0 trois\\n                            parents\\u00a0\\u00bb\\u00a0: l\\u2019ADN nucl\\u00e9aire du b\\u00e9b\\u00e9 vient des deux parents,\\n                            alors que l\\u2019ADN mitochondrial vient d\\u2019une troisi\\u00e8me personne. En 2000,\\n                            Sharon Saarinen, de West Bloomfield, dans le Michigan, a donn\\u00e9 naissance\\n                            \\u00e0 une fille en bonne sant\\u00e9, Alana\\u00a0: son ADN nucl\\u00e9aire vient de sa\\n                            m\\u00e8re Sharon et de son p\\u00e8re, Paul, mais son ADN mitochondrial est celui\\n                            d\\u2019une autre femme. D\\u2019un point de vue purement technique, Alana a trois\\n                            parents biologiques. Un an plus tard, en 2001, le gouvernement am\\u00e9ricain\\n                            a interdit cette proc\\u00e9dure pour des raisons de s\\u00e9curit\\u00e9 et\\n                                d\\u2019\\u00e9thique(50).", "cluster": 255}, {"text": "Le 3\\u00a0f\\u00e9vrier 2015, le Parlement britannique a adopt\\u00e9 une loi sur\\n                            l\\u2019embryon \\u00ab\\u00a0de trois parents\\u00a0\\u00bb, autorisant cette proc\\u00e9dure\\n                            \\u2013\\u00a0et la recherche qui l\\u2019accompagne\\u00a0\\u2013 au Royaume-Uni(51).\\n                            \\u00c0 l\\u2019heure actuelle, il est techniquement impossible, et ill\\u00e9gal, de\\n                            remplacer l\\u2019ADN nucl\\u00e9aire, mais si les difficult\\u00e9s techniques sont un\\n                            jour r\\u00e9solues, la logique qui a pr\\u00e9sid\\u00e9 au remplacement de l\\u2019ADN\\n                            mitochondrial d\\u00e9faillant pourrait justifier d\\u2019en faire autant avec l\\u2019ADN\\n                            nucl\\u00e9aire.", "cluster": 255}, {"text": "Apr\\u00e8s la s\\u00e9lection et le remplacement, la prochaine \\u00e9tape en puissance\\n                            est l\\u2019amendement. Du jour o\\u00f9 il sera possible d\\u2019amender les g\\u00e8nesmortels, pourquoi se donner la peine d\\u2019ins\\u00e9rer de l\\u2019ADN\\n                            \\u00e9tranger, alors qu\\u2019il suffit de r\\u00e9\\u00e9crire le code et de transformer un\\n                            g\\u00e8ne mutant dangereux en une version b\\u00e9nigne\\u00a0? Nous pourrions\\n                            ensuite, selon le m\\u00eame m\\u00e9canisme, r\\u00e9parer non seulement des g\\u00e8nes\\n                            l\\u00e9taux, mais aussi les g\\u00e8nes responsables de maladies moins graves, de\\n                            l\\u2019autisme, du cr\\u00e9tinisme ou de l\\u2019ob\\u00e9sit\\u00e9. Qui voudrait que son enfant\\n                            souffre de l\\u2019une d\\u2019elles\\u00a0? Supposez qu\\u2019un test g\\u00e9n\\u00e9tique indique\\n                            que votre fille sera tr\\u00e8s probablement intelligente, belle et g\\u00e9n\\u00e9reuse,\\n                            mais souffrira de d\\u00e9pression chronique. Ne voudriez-vous pas lui\\n                            \\u00e9pargner des ann\\u00e9es de malheur par une intervention rapide et indolore\\n                            en \\u00e9prouvette\\u00a0?", "cluster": 252}, {"text": "Et, tant que vous y \\u00eates, pourquoi ne pas donner \\u00e0 l\\u2019enfant un petit coup\\n                            de pouce\\u00a0? La vie est rude et \\u00e9prouvante, m\\u00eame pour les gens en\\n                            bonne sant\\u00e9. Il serait donc bien utile que la fillette poss\\u00e8de un\\n                            syst\\u00e8me immunitaire plus robuste que la normale, une m\\u00e9moire sup\\u00e9rieure\\n                            \\u00e0 la moyenne ou un temp\\u00e9rament particuli\\u00e8rement radieux. Et si vous ne\\n                            voulez pas de cela pour votre enfant\\u2026 mais que vos voisins le fassent\\n                            pour les leurs\\u00a0? Le laisseriez-vous \\u00eatre \\u00e0 la tra\\u00eene\\u00a0? Et si\\n                            le gouvernement interdisait \\u00e0 tous ses citoyens ce genre de\\n                            manipulation, et que les Nord-Cor\\u00e9ens les pratiquaient et produisaient\\n                            des g\\u00e9nies, des artistes et des athl\\u00e8tes \\u00e9blouissants bien sup\\u00e9rieurs\\n                            aux n\\u00f4tres\\u00a0? Voici comment, \\u00e0 petits pas, nous progressons sur la\\n                            voie d\\u2019un catalogue d\\u2019enfants g\\u00e9n\\u00e9tiques.", "cluster": 252}, {"text": "Soigner est la justification initiale de toute am\\u00e9lioration. Demandez aux\\n                            professeurs qui exp\\u00e9rimentent le g\\u00e9nie g\\u00e9n\\u00e9tique ou les interfaces\\n                            cerveau-ordinateur pourquoi ils sont engag\\u00e9s dans ces recherches. Tr\\u00e8s\\n                            probablement r\\u00e9pondront-ils qu\\u2019ils le font pour gu\\u00e9rir une\\n                            maladie\\u00a0: \\u00ab\\u00a0\\u00c0 l\\u2019aide du g\\u00e9nie g\\u00e9n\\u00e9tique, nous pourrons\\n                            vaincre le cancer. Et si nous pouvions connecter directement le cerveau\\n                            \\u00e0 un ordinateur, nous pourrions gu\\u00e9rir la schizophr\\u00e9nie.\\u00a0\\u00bb\\n                            Peut-\\u00eatre bien, mais \\u00e7a ne s\\u2019arr\\u00eatera s\\u00fbrement pas l\\u00e0. Quand nous serons\\n                            parvenus \\u00e0 connecter cerveaux et ordinateurs, n\\u2019emploierons-nous cette\\n                            technique que pour soigner la schizophr\\u00e9nie\\u00a0? Si quelqu\\u2019un a la\\n                            na\\u00efvet\\u00e9 d\\u2019y croire, sans doute en conna\\u00eet-il un rayon sur les cerveaux\\n                            et les ordinateurs, mais beaucoup moins sur la psych\\u00e9 et la soci\\u00e9t\\u00e9\\n                            humaines. Sit\\u00f4t qu\\u2019a \\u00e9t\\u00e9 op\\u00e9r\\u00e9e une perc\\u00e9e capitale, il est impossible\\n                            de la cantonner \\u00e0 la gu\\u00e9rison, et d\\u2019interdire formellement qu\\u2019elle soit\\n                            employ\\u00e9e pour l\\u2019am\\u00e9lioration.", "cluster": -1}, {"text": "Bien entendu, les humains peuvent limiter et limitent\\n                            leur usage des nouvelles technologies. Le mouvement eug\\u00e9niste est tomb\\u00e9\\n                            en disgr\\u00e2ce apr\\u00e8s la Seconde Guerre mondiale et, bien que le commerce\\n                            d\\u2019organes soit d\\u00e9sormais possible et potentiellement tr\\u00e8s lucratif, il\\n                            est demeur\\u00e9 jusqu\\u2019ici une activit\\u00e9 p\\u00e9riph\\u00e9rique. Si proposer des b\\u00e9b\\u00e9s \\u00e0\\n                            la carte pourrait bien devenir techniquement aussi r\\u00e9alisable que tuer\\n                            pour r\\u00e9cup\\u00e9rer les organes de ses victimes, ces deux activit\\u00e9s restent\\n                            pour l\\u2019heure marginales.", "cluster": -1}, {"text": "De m\\u00eame que nous avons \\u00e9chapp\\u00e9 aux griffes de la loi de Tchekhov pour ce\\n                            qui est de la guerre, nous pouvons aussi leur \\u00e9chapper dans d\\u2019autres\\n                            champs d\\u2019action. Certaines armes apparaissent sur sc\\u00e8ne sans que jamais\\n                            un coup de feu soit tir\\u00e9. C\\u2019est bien pourquoi il est vital de r\\u00e9fl\\u00e9chir\\n                            au nouvel ordre du jour de l\\u2019humanit\\u00e9. Pr\\u00e9cis\\u00e9ment parce que nous avons\\n                            le choix concernant l\\u2019usage des nouvelles technologies, mieux vaut\\n                            comprendre ce qui se passe et d\\u00e9cider avant qu\\u2019elles ne d\\u00e9cident pour\\n                            nous.", "cluster": 32}, {"text": "La pr\\u00e9diction selon laquelle auXXIesi\\u00e8cle\\n                            l\\u2019humanit\\u00e9 est susceptible de viser l\\u2019immortalit\\u00e9, la f\\u00e9licit\\u00e9 et la\\n                            divinit\\u00e9 risque de f\\u00e2cher, d\\u2019ali\\u00e9ner ou d\\u2019effrayer un certain nombre de\\n                            gens, et quelques \\u00e9claircissements s\\u2019imposent.", "cluster": 258}, {"text": "Premi\\u00e8rement, il ne s\\u2019agit pas de ce que feront la plupart des individus\\n                            auXXIesi\\u00e8cle, mais de ce que fera l\\u2019humanit\\u00e9\\n                            collectivement. La tr\\u00e8s grande majorit\\u00e9 ne jouera probablement, au\\n                            mieux, qu\\u2019un r\\u00f4le mineur dans ces projets. M\\u00eame si la famine, les\\n                            \\u00e9pid\\u00e9mies et la guerre r\\u00e9gressent, des milliards d\\u2019\\u00eatres humains, dans\\n                            les pays en voie de d\\u00e9veloppement et les quartiers mis\\u00e9rables,\\n                            continueront d\\u2019affronter la pauvret\\u00e9, la maladie et la violence, alors\\n                            m\\u00eame que les \\u00e9lites recherchent d\\u00e9j\\u00e0 la jeunesse \\u00e9ternelle et des\\n                            pouvoirs divins. L\\u2019injustice semble flagrante. Tant qu\\u2019un seul enfant\\n                            meurt encore de malnutrition ou qu\\u2019un adulte trouve la mort dans la\\n                            guerre entre narcotrafiquants, pourrait-on plaider, l\\u2019humanit\\u00e9 devrait\\n                            tout faire pour combattre ces fl\\u00e9aux. Ce n\\u2019est que lorsque la derni\\u00e8re\\n                            \\u00e9p\\u00e9e aura \\u00e9t\\u00e9 transform\\u00e9e en soc de charrue quenous\\n                            devrions nous consacrer au grand enjeu suivant. Mais l\\u2019histoire ne\\n                            fonctionne pas comme \\u00e7a. Les desseins de ceux qui vivent dans les palais\\n                            ont toujours \\u00e9t\\u00e9 diff\\u00e9rents de ceux des habitants des quartiers\\n                            d\\u00e9favoris\\u00e9s, et il y a peu de chances que cela change auXXIesi\\u00e8cle.", "cluster": -1}, {"text": "Deuxi\\u00e8mement, ce tableau n\\u2019est pas un manifeste politique, mais une\\n                            pr\\u00e9diction historique. Ind\\u00e9pendamment du sort des bidonvilles, il est\\n                            loin d\\u2019\\u00eatre \\u00e9vident qu\\u2019il faille viser l\\u2019immortalit\\u00e9, le bonheur et la\\n                            divinit\\u00e9. Adopter ces projets pourrait bien \\u00eatre une grave erreur. Mais\\n                            l\\u2019histoire fourmille de graves erreurs. Compte tenu de notre pass\\u00e9 et de\\n                            nos valeurs actuelles, nous risquons cependant de nous lancer dans cette\\n                            qu\\u00eate, m\\u00eame si elle doit nous tuer.", "cluster": -1}, {"text": "Troisi\\u00e8mement, chercher ne veut pas dire r\\u00e9ussir. L\\u2019histoire est souvent\\n                            fa\\u00e7onn\\u00e9e par des espoirs d\\u00e9mesur\\u00e9s. L\\u2019histoire russe duXXesi\\u00e8cle a \\u00e9t\\u00e9 largement fa\\u00e7onn\\u00e9e par les\\n                            efforts communistes pour vaincre les in\\u00e9galit\\u00e9s, mais ils ont \\u00e9chou\\u00e9. Ma\\n                            pr\\u00e9diction met l\\u2019accent sur ce que l\\u2019humanit\\u00e9 vaessayerd\\u2019accomplir auXXIesi\\u00e8cle, non pas sur ce qu\\u2019elleparviendra\\u00e0 accomplir. L\\u2019effort pour triompher de la mort\\n                            fa\\u00e7onnera l\\u2019\\u00e9conomie, la soci\\u00e9t\\u00e9 et la politique futures. Il ne s\\u2019ensuit\\n                            pas qu\\u2019en 2100 les hommes seront immortels.", "cluster": -1}, {"text": "Quatri\\u00e8mement, et c\\u2019est le plus important, cette pr\\u00e9diction est moins une\\n                            proph\\u00e9tie qu\\u2019une fa\\u00e7on de d\\u00e9battre de nos choix pr\\u00e9sents. Si la\\n                            discussion nous m\\u00e8ne \\u00e0 d\\u2019autres choix, et que la pr\\u00e9diction se r\\u00e9v\\u00e8le\\n                            fausse, tant mieux. \\u00c0 quoi bon en faire si elles n\\u2019ont pas\\n                            d\\u2019effets\\u00a0?", "cluster": -1}]'

In [9]:
kg = invoke({"chunks": chunks})

Parsing error: Failed to parse EssayGraph from completion {"concepts": [{"id": 1, "summary": "Humanity's recent achievements have fueled ambition, leading to increasingly audacious goals.", "title": "Human Ambition"}, {"id": 2, "summary": "The pursuit of immortality is a natural progression from past successes in combating hunger, disease, and violence.", "title": "The Pursuit of Immortality"}, {"id": 3, "summary": "Modern culture values human life above all else, making death a crime against humanity.", "title": "Modern Culture's Value of Human Life"}, {"id": 4, "summary": "Traditional religions and ideologies tolerated death, as they focused on an afterlife or a higher purpose beyond earthly existence.", "title": "Traditional Religions and Death"}, {"id": 5, "summary": "Modern science and culture view death as a technical problem that can and should be solved.", "title": "Modern Science and Death"}, {"id": 6, "summary": "Death is a technical failure caused by biological malfunctions,

In [12]:
create_knowledge_graph(kg.dict())

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/blollivier/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [13]:
kg


EssayGraph(
    concepts=[
        Concept(
            id=1,
            summary="Humanity's recent achievements have fueled ambition, leading to increasingly audacious goals.",
            title='Human Ambition'
        ),
        Concept(
            id=2,
            summary='The pursuit of immortality is a natural progression from past successes in combating hunger, disease, and violence.',
            title='The Pursuit of Immortality'
        ),
        Concept(
            id=3,
            summary='Modern culture values human life above all else, making death a crime against humanity.',
            title="Modern Culture's Value of Human Life"
        ),
        Concept(
            id=4,
            summary='Traditional religions and ideologies tolerated death, as they focused on an afterlife or a higher purpose beyond earthly existence.',
            title='Traditional Religions and Death'
        ),
        Concept(
            id=5,
            summary='Modern science and 